# <center> Model Gini Calculation </center>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

from sklearn.metrics import roc_auc_score
from datetime import datetime, timedelta
# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

# Function

## calculate_gini_for_threedigitscore

In [2]:
# def calculate_gini_for_threedigitscore(scores, labels):
#     """
#     Calculate Gini coefficient for three-digit scores and binary labels
    
#     Parameters:
#     scores: array-like, three-digit scores (higher is better)
#     labels: array-like, binary values (0 or 1, where 1 indicates default)
    
#     Returns:
#     float: Gini coefficient
#     """
#     # Combine scores and labels into a DataFrame
#     df = pd.DataFrame({'score': scores, 'label': labels})
    
#     # Sort by score in descending order (assuming higher score is better)
#     df = df.sort_values('score', ascending=False)
    
#     # Calculate cumulative values
#     total_pos = df['label'].sum()
#     total_neg = len(df) - total_pos
    
#     if total_pos == 0 or total_neg == 0:
#         return 0
    
#     # Calculate cumulative proportions
#     cum_pos = df['label'].cumsum()
#     cum_neg = np.arange(1, len(df) + 1) - cum_pos
    
#     # Convert to proportions
#     cum_pos_prop = cum_pos / total_pos
#     cum_neg_prop = cum_neg / total_neg
    
#     # Calculate Gini
#     gini = 1 - np.trapz(cum_pos_prop, cum_neg_prop)
    
#     return gini


## Modified one

def calculate_gini_for_threedigitscore(scores, labels):
    """
    Calculate Gini coefficient for three-digit scores and binary labels
    
    Parameters:
    scores: array-like, three-digit scores (higher is better)
    labels: array-like, binary values (0 or 1, where 1 indicates default)
    
    Returns:
    float: Gini coefficient
    """
    # Combine scores and labels into a DataFrame
    df = pd.DataFrame({'score': scores, 'label': labels})
    
    # Sort by score in descending order (assuming higher score means lower risk)
    # For default prediction, we want to sort scores in ascending order 
    # since higher default probability should correspond to higher risk
    df = df.sort_values('score', ascending=True)  # Changed to ascending=True
    
    # Calculate cumulative values
    total_pos = df['label'].sum()
    total_neg = len(df) - total_pos
    
    if total_pos == 0 or total_neg == 0:
        return 0
    
    # Calculate cumulative proportions
    cum_pos = df['label'].cumsum()
    cum_neg = np.arange(1, len(df) + 1) - cum_pos
    
    # Convert to proportions
    cum_pos_prop = cum_pos / total_pos
    cum_neg_prop = cum_neg / total_neg
    
    # Calculate area under curve
    auc = np.trapz(cum_pos_prop, cum_neg_prop)
    
    # Calculate Gini
    gini = 2 * auc - 1
    
    return gini

## calculate_gini

In [3]:
def calculate_gini(pd_scores, bad_indicators):
    """
    Calculate Gini coefficient from scores and binary indicators
    
    Parameters:
    pd_scores: array-like of scores/probabilities
    bad_indicators: array-like of binary outcomes (0/1)
    
    Returns:
    float: Gini coefficient
    """
    # Convert inputs to numpy arrays and ensure they're numeric
    pd_scores = np.array(pd_scores, dtype=float)
    bad_indicators = np.array(bad_indicators, dtype=int)
    
    # Check for valid input data
    if len(pd_scores) == 0 or len(bad_indicators) == 0:
        return np.nan
    
    # Check if we have both good and bad cases (needed for ROC AUC)
    if len(np.unique(bad_indicators)) < 2:
        return np.nan
    
    # Calculate AUC using sklearn
    try:
        auc = roc_auc_score(bad_indicators, pd_scores)
        # Calculate Gini from AUC
        gini = 2 * auc - 1
        return gini
    except ValueError:
        return np.nan

## calculate_hybrid_gini

In [4]:
# def calculate_hybrid_gini(scores, labels):
#     """
#     Calculate Gini coefficient handling both PD values and three-digit scores
    
#     Parameters:
#     scores: array-like, contains either PD values (0-1) or three-digit scores
#     labels: array-like, binary values (0 or 1, where 1 indicates default)
    
#     Returns:
#     float: Gini coefficient
#     """
#     # Convert inputs to numpy arrays
#     scores = np.array(scores, dtype=float)
#     labels = np.array(labels, dtype=int)
    
#     # Basic validation
#     if len(scores) == 0 or len(labels) == 0:
#         return np.nan
    
#     if len(np.unique(labels)) < 2:
#         return np.nan
        
#     # Determine if scores are PD values or three-digit scores
#     # PD values are between 0 and 1
#     is_pd = np.all((scores >= 0) & (scores <= 1))
    
#     if is_pd:
#         try:
#             auc = roc_auc_score(labels, scores)
#             gini = 2 * auc - 1
#             return gini
#         except ValueError:
#             return np.nan
#     else:
#         # Handle as three-digit score
#         df = pd.DataFrame({'score': scores, 'label': labels})
#         df = df.sort_values('score', ascending=False)
        
#         total_pos = df['label'].sum()
#         total_neg = len(df) - total_pos
        
#         if total_pos == 0 or total_neg == 0:
#             return np.nan
        
#         cum_pos = df['label'].cumsum()
#         cum_neg = np.arange(1, len(df) + 1) - cum_pos
        
#         cum_pos_prop = cum_pos / total_pos
#         cum_neg_prop = cum_neg / total_neg
        
#         gini = 1 - np.trapz(cum_pos_prop, cum_neg_prop)
#         return gini

## Modified one

def calculate_hybrid_gini(scores, labels):
    """
    Calculate Gini coefficient handling both PD values and three-digit scores
    
    Parameters:
    scores: array-like, contains either PD values (0-1) or three-digit scores
    labels: array-like, binary values (0 or 1, where 1 indicates default)
    
    Returns:
    float: Gini coefficient
    """
    # Convert inputs to numpy arrays
    scores = np.array(scores, dtype=float)
    labels = np.array(labels, dtype=int)
    
    # Basic validation
    if len(scores) == 0 or len(labels) == 0:
        return np.nan
    
    if len(np.unique(labels)) < 2:
        return np.nan
        
    # Determine if scores are PD values or three-digit scores
    # PD values are between 0 and 1
    is_pd = np.all((scores >= 0) & (scores <= 1))
    
    if is_pd:
        try:
            auc = roc_auc_score(labels, scores)
            gini = 2 * auc - 1
            return gini
        except ValueError:
            return np.nan
    else:
        # Handle as three-digit score
        df = pd.DataFrame({'score': scores, 'label': labels})
        # Sort by score in ascending order since higher score means higher risk
        df = df.sort_values('score', ascending=True)
        
        total_pos = df['label'].sum()
        total_neg = len(df) - total_pos
        
        if total_pos == 0 or total_neg == 0:
            return np.nan
        
        cum_pos = df['label'].cumsum()
        cum_neg = np.arange(1, len(df) + 1) - cum_pos
        
        cum_pos_prop = cum_pos / total_pos
        cum_neg_prop = cum_neg / total_neg
        
        # Calculate area under curve
        auc = np.trapz(cum_pos_prop, cum_neg_prop)
        
        # Calculate Gini using the same formula as PD values
        gini = 2 * auc - 1
        return gini

## calculate_periodic_gini_threedigit

In [5]:
# Main processing code
def calculate_periodic_gini_threedigit(df, score_column, label_column, namecolumn):
    """
    Calculate periodic Gini coefficients
    
    Parameters:
    df: DataFrame with disbursement dates and score/label columns
    score_column: name of the score column
    label_column: name of the label column
    """
    # Ensure date is datetime type
    df['disbursementdate'] = pd.to_datetime(df['disbursementdate'])
    
    # Calculate weekly Gini
    df['week'] = df['disbursementdate'].dt.to_period('W')
    weekly_gini = df.groupby('week').apply(
        lambda x: calculate_gini_for_threedigitscore(x[score_column], x[label_column])
    ).reset_index(name='gini')
    weekly_gini['period'] = 'Week'
    weekly_gini['start_date'] = weekly_gini['week'].apply(lambda x: x.to_timestamp())
    weekly_gini['end_date'] = weekly_gini['start_date'] + timedelta(days=6)
    weekly_gini = weekly_gini[['start_date', 'end_date', 'gini', 'period']]
    
    # Calculate monthly Gini
    df['month'] = df['disbursementdate'].dt.to_period('M')
    monthly_gini = df.groupby('month').apply(
        lambda x: calculate_gini_for_threedigitscore(x[score_column], x[label_column])
    ).reset_index(name='gini')
    monthly_gini['period'] = 'Month'
    monthly_gini['start_date'] = monthly_gini['month'].apply(lambda x: x.to_timestamp())
    monthly_gini['end_date'] = monthly_gini['start_date'] + pd.DateOffset(months=1) - pd.Timedelta(days=1)
    monthly_gini = monthly_gini[['start_date', 'end_date', 'gini', 'period']]
    
    # Combine and sort results
    gini_results = pd.concat([weekly_gini, monthly_gini])
    gini_results = gini_results.sort_values(by='start_date').reset_index(drop=True)
    
    # Add metadata columns
    gini_results['Model_Name'] = score_column
    gini_results['version'] = '1.1.0'
    gini_results['bad_rate'] = namecolumn
    gini_results.rename(columns={'gini': f'{score_column}_{namecolumn}_gini'}, inplace=True)
    
    return gini_results

## calculate_periodic_gini

In [6]:
def calculate_periodic_gini(df, score_column, label_column, namecolumn):
    """
    Calculate periodic Gini coefficients
    
    Parameters:
    df: DataFrame with disbursement dates and score/label columns
    score_column: name of the score column
    label_column: name of the label column
    """
    # Input validation
    required_columns = ['disbursementdate', score_column, label_column]
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"Missing required columns. Need: {required_columns}")
        
    # Create a copy to avoid modifying original dataframe
    df = df.copy()
    
    # Ensure date is datetime type
    df['disbursementdate'] = pd.to_datetime(df['disbursementdate'])
    
    # Ensure score and label columns are numeric
    df[score_column] = pd.to_numeric(df[score_column], errors='coerce')
    df[label_column] = pd.to_numeric(df[label_column], errors='coerce')
    
    # Drop rows with invalid values
    df = df.dropna(subset=[score_column, label_column])
    
    # Calculate weekly Gini
    df['week'] = df['disbursementdate'].dt.to_period('W')
    weekly_gini = df.groupby('week').apply(
        lambda x: calculate_gini(x[score_column], x[label_column])
        if len(x) >= 10 else np.nan  # Only calculate if we have enough samples
    ).reset_index(name='gini')
    weekly_gini['period'] = 'Week'
    weekly_gini['start_date'] = weekly_gini['week'].apply(lambda x: x.to_timestamp())
    weekly_gini['end_date'] = weekly_gini['start_date'] + timedelta(days=6)
    weekly_gini = weekly_gini[['start_date', 'end_date', 'gini', 'period']]
    
    # Calculate monthly Gini
    df['month'] = df['disbursementdate'].dt.to_period('M')
    monthly_gini = df.groupby('month').apply(
        lambda x: calculate_gini(x[score_column], x[label_column])
        if len(x) >= 20 else np.nan  # Only calculate if we have enough samples
    ).reset_index(name='gini')
    monthly_gini['period'] = 'Month'
    monthly_gini['start_date'] = monthly_gini['month'].apply(lambda x: x.to_timestamp())
    monthly_gini['end_date'] = monthly_gini['start_date'] + pd.DateOffset(months=1) - pd.Timedelta(days=1)
    monthly_gini = monthly_gini[['start_date', 'end_date', 'gini', 'period']]
    
    # Combine and sort results
    gini_results = pd.concat([weekly_gini, monthly_gini])
    gini_results = gini_results.sort_values(by='start_date').reset_index(drop=True)
    
    # Add metadata columns
    gini_results['Model_Name'] = score_column
    gini_results['version'] = '1.1.0'
    gini_results['bad_rate'] = namecolumn
    gini_results.rename(columns={'gini': f'{score_column}_{namecolumn}_gini'}, inplace=True)
    
    return gini_results

## calculate_periodic_hybrid_gini

In [7]:
def calculate_periodic_hybrid_gini(df, score_column, label_column, namecolumn):
    """
    Calculate periodic Gini coefficients for mixed score types
    
    Parameters:
    df: DataFrame with disbursement dates and score/label columns
    score_column: name of the score column
    label_column: name of the label column
    """
    # Input validation
    required_columns = ['disbursementdate', score_column, label_column]
    if not all(col in df.columns for col in required_columns):
        raise ValueError(f"Missing required columns. Need: {required_columns}")
        
    # Create a copy to avoid modifying original dataframe
    df = df.copy()
    
    # Ensure date is datetime type
    df['disbursementdate'] = pd.to_datetime(df['disbursementdate'])
    
    # Ensure score and label columns are numeric
    df[score_column] = pd.to_numeric(df[score_column], errors='coerce')
    df[label_column] = pd.to_numeric(df[label_column], errors='coerce')
    
    # Drop rows with invalid values
    df = df.dropna(subset=[score_column, label_column])
    
    # Calculate weekly Gini
    df['week'] = df['disbursementdate'].dt.to_period('W')
    weekly_gini = df.groupby('week').apply(
        lambda x: calculate_hybrid_gini(x[score_column], x[label_column])
        if len(x) >= 10 else np.nan  # Only calculate if we have enough samples
    ).reset_index(name='gini')
    weekly_gini['period'] = 'Week'
    weekly_gini['start_date'] = weekly_gini['week'].apply(lambda x: x.to_timestamp())
    weekly_gini['end_date'] = weekly_gini['start_date'] + pd.Timedelta(days=6)
    weekly_gini = weekly_gini[['start_date', 'end_date', 'gini', 'period']]
    
    # Calculate monthly Gini
    df['month'] = df['disbursementdate'].dt.to_period('M')
    monthly_gini = df.groupby('month').apply(
        lambda x: calculate_hybrid_gini(x[score_column], x[label_column])
        if len(x) >= 20 else np.nan  # Only calculate if we have enough samples
    ).reset_index(name='gini')
    monthly_gini['period'] = 'Month'
    monthly_gini['start_date'] = monthly_gini['month'].apply(lambda x: x.to_timestamp())
    monthly_gini['end_date'] = monthly_gini['start_date'] + pd.DateOffset(months=1) - pd.Timedelta(days=1)
    monthly_gini = monthly_gini[['start_date', 'end_date', 'gini', 'period']]
    
    # Combine and sort results
    gini_results = pd.concat([weekly_gini, monthly_gini])
    gini_results = gini_results.sort_values(by='start_date').reset_index(drop=True)
    
    # Add metadata columns
    gini_results['Model_Name'] = score_column
    gini_results['version'] = '1.1.0'
    gini_results['bad_rate'] = namecolumn
    gini_results.rename(columns={'gini': f'{score_column}_{namecolumn}_gini'}, inplace=True)
    
    return gini_results

# App Score FPD10

In [9]:
sq = """with appscore as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    apps_score s_apps_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    apps_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from appscore;"""

dfappscorefpd10 = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID ece40ffb-d255-4884-a606-37512642eab2 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [10]:
dfappscorefpd10.sample(5)

,disbursementdate,digitalLoanAccountId,s_apps_score,ln_fpd10_flag,ln_mature_fpd10_flag
14407,2024-11-16 09:50:59,20cf42e0-1abd-491f-bc2d-372661e9d2c1,0.452690,0,1
162193,2024-05-07 12:14:36,2dc4f4ed-aecd-4a64-b53d-75f88692ee55,0.623222,0,1
26169,2025-01-03 13:37:24,f67ef276-19ef-4888-971e-3b1e4b44e8a1,0.610334,0,1
78533,2024-11-10 16:48:45,e1658eb9-6d76-4813-9c20-dec47ff6b7a7,0.414407,0,1
76120,2024-07-16 17:16:50,ad4c3304-36a9-4b44-a6c6-a9a712b4d4c1,0.457169,0,1


In [11]:
gini_results = calculate_periodic_gini(dfappscorefpd10, 's_apps_score', 'ln_fpd10_flag', 'FPD10')

In [12]:
gini_results.head()

,start_date,end_date,s_apps_score_FPD10_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.548230,Week,s_apps_score,1.1.0,FPD10
1,2023-06-01,2023-06-30,0.384648,Month,s_apps_score,1.1.0,FPD10
2,2023-06-05,2023-06-11,0.500000,Week,s_apps_score,1.1.0,FPD10
3,2023-06-12,2023-06-18,0.555195,Week,s_apps_score,1.1.0,FPD10
4,2023-06-19,2023-06-25,0.004831,Week,s_apps_score,1.1.0,FPD10


In [13]:
appscoreFPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{appscoreFPD10.shape}")
appscoreFPD10.columns.values

The shape of dataframe after copy is:	(108, 7)


array(['start_date', 'end_date', 's_apps_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [14]:
gini_results.head()

,start_date,end_date,s_apps_score_FPD10_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.548230,Week,s_apps_score,1.1.0,FPD10
1,2023-06-01,2023-06-30,0.384648,Month,s_apps_score,1.1.0,FPD10
2,2023-06-05,2023-06-11,0.500000,Week,s_apps_score,1.1.0,FPD10
3,2023-06-12,2023-06-18,0.555195,Week,s_apps_score,1.1.0,FPD10
4,2023-06-19,2023-06-25,0.004831,Week,s_apps_score,1.1.0,FPD10


# App Score FPD30

In [15]:
sq = """
with appscore as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    apps_score s_apps_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    apps_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from appscore;
"""

dfappscorefpd30 = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID 9a5b3864-a8ac-4bf3-9f21-2dab53aea571 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [16]:
dfappscorefpd30.head()

,disbursementdate,digitalLoanAccountId,s_apps_score,ln_fpd30_flag,ln_mature_fpd30_flag
0,2023-06-10 16:10:58,361d32b4-00b3-4ac6-a9a0-99e1481c1d0c,0.410029,0,1
1,2023-06-13 11:53:20,2b68c9d3-5a5c-4f09-be5f-b6a76c13cd31,0.626895,0,1
2,2023-06-13 12:05:25,64fe2c12-ff01-4f3b-9730-fe13f42787e8,0.404328,0,1
3,2023-06-04 14:08:26,2b0786f2-4b06-4209-9fc0-cf2506d2306c,0.384938,0,1
4,2023-06-13 17:23:51,68da4d62-08c4-4eb9-87f3-bece188fc077,0.481916,0,1


In [17]:
gini_results = calculate_periodic_gini(dfappscorefpd30, 's_apps_score', 'ln_fpd30_flag', 'FPD30')
# gini_results['bad_rate'] = 'FPD30'
appscoreFPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{appscoreFPD30.shape}")
appscoreFPD30.columns.values

The shape of dataframe after copy is:	(102, 7)


array(['start_date', 'end_date', 's_apps_score_FPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [19]:
gini_results.sample(5)

,start_date,end_date,s_apps_score_FPD30_gini,period,Model_Name,version,bad_rate
77,2024-08-05,2024-08-11,0.298875,Week,s_apps_score,1.1.0,FPD30
57,2024-04-15,2024-04-21,0.416764,Week,s_apps_score,1.1.0,FPD30
72,2024-07-08,2024-07-14,0.319215,Week,s_apps_score,1.1.0,FPD30
64,2024-05-27,2024-06-02,0.438785,Week,s_apps_score,1.1.0,FPD30
83,2024-09-09,2024-09-15,0.285449,Week,s_apps_score,1.1.0,FPD30


# App Score FSPD30

In [20]:
sq = """with appscore as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    apps_score s_apps_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    apps_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from appscore;
"""

dfappscorefspd30 = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID 2fdf4c3c-ae52-42e8-8c46-0e766542419b successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [21]:
gini_results = calculate_periodic_gini(dfappscorefspd30, 's_apps_score', 'ln_fspd30_flag', 'FSPD30')
# gini_results['bad_rate'] = 'FSPD30'
appscoreFSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{appscoreFSPD30.shape}")
appscoreFSPD30.columns.values

The shape of dataframe after copy is:	(100, 7)


array(['start_date', 'end_date', 's_apps_score_FSPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [22]:
gini_results.tail()

,start_date,end_date,s_apps_score_FSPD30_gini,period,Model_Name,version,bad_rate
95,2024-11-18,2024-11-24,0.302444,Week,s_apps_score,1.1.0,FSPD30
96,2024-11-25,2024-12-01,0.260883,Week,s_apps_score,1.1.0,FSPD30
97,2024-12-01,2024-12-31,0.311393,Month,s_apps_score,1.1.0,FSPD30
98,2024-12-02,2024-12-08,0.309329,Week,s_apps_score,1.1.0,FSPD30
99,2024-12-09,2024-12-15,0.261535,Week,s_apps_score,1.1.0,FSPD30


# App Score FSTPD30

In [23]:
sq = """with appscore as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    apps_score s_apps_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    apps_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from appscore;
"""

dfappscorefstpd30 = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID d0eda80d-e728-4a2f-a230-f2e0e0b21bb2 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [24]:
gini_results = calculate_periodic_gini(dfappscorefstpd30, 's_apps_score', 'ln_fstpd30_flag', 'FSTPD30')
# gini_results['bad_rate'] = 'FSTPD30'
appscoreFSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{appscoreFSTPD30.shape}")
appscoreFSTPD30.columns.values

The shape of dataframe after copy is:	(94, 7)


array(['start_date', 'end_date', 's_apps_score_FSTPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [25]:
appscoreFSTPD30.head()

,start_date,end_date,s_apps_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.465098,Week,s_apps_score,1.1.0,FSTPD30
1,2023-06-01,2023-06-30,0.337420,Month,s_apps_score,1.1.0,FSTPD30
2,2023-06-05,2023-06-11,0.413300,Week,s_apps_score,1.1.0,FSTPD30
3,2023-06-12,2023-06-18,0.395676,Week,s_apps_score,1.1.0,FSTPD30
4,2023-06-19,2023-06-25,0.314497,Week,s_apps_score,1.1.0,FSTPD30


# Combining App Score

In [26]:
import functools

dataframes = [appscoreFPD10, appscoreFPD30, appscoreFSPD30, appscoreFSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 's_apps_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 's_apps_score_FPD30_gini',
       's_apps_score_FSPD30_gini', 's_apps_score_FSTPD30_gini'],
      dtype=object)

In [27]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'bad_rate','s_apps_score_FSTPD30_gini','s_apps_score_FSPD30_gini',
       's_apps_score_FPD30_gini', 's_apps_score_FPD10_gini']].copy()
final_df.dtypes

start_date                   datetime64[ns]
end_date                     datetime64[ns]
period                               object
Model_Name                           object
version                              object
bad_rate                             object
s_apps_score_FSTPD30_gini           float64
s_apps_score_FSPD30_gini            float64
s_apps_score_FPD30_gini             float64
s_apps_score_FPD10_gini             float64
dtype: object

## Creating app score table 

In [28]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_s_apps_score;"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=2d2a1f3e-162b-4f86-ac0d-c588b6c5438d>

In [29]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('s_apps_score_FSTPD30_gini', 'FLOAT'),
    bigquery.SchemaField('s_apps_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('s_apps_score_FPD30_gini', 'FLOAT'),
    bigquery.SchemaField('s_apps_score_fpd10_gini', 'FLOAT')
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_s_apps_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=7b5c243b-5ea9-46ab-9bf4-17e7a2946c1a>

# sb_demo_score

## FPD10

In [31]:
sq = """
with sb_demo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    beta_demo_score sb_demo_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    beta_demo_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from sb_demo_score;
"""

df_sb_demo_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')





Job ID 0149e231-9cd7-4404-92e3-6c1bd1726b38 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(




Query is running:   0%|          |



Downloading: 100%|██████████|


In [32]:
df_sb_demo_scorefpd10.head()

,disbursementdate,digitalLoanAccountId,sb_demo_score,ln_fpd10_flag,ln_mature_fpd10_flag
0,2024-09-01 15:18:37,1a2cb6f2-e66f-444a-ba22-a05cb5652ef2,0.085835,0,1
1,2024-05-02 17:24:11,9b118965-74ea-4832-a640-8b5ea2cc140e,0.088430,0,1
2,2024-09-19 20:32:20,4f7bb0f5-ec5b-40c0-a145-6d8aa9b083ee,0.071088,1,1
3,2024-06-19 20:08:28,210afa02-21e2-4ecc-8e28-06969222c968,0.155124,0,1
4,2025-01-27 12:33:16,107d411b-cd24-4dde-86a8-6f7eeeaaa7bb,0.031791,0,1


In [33]:
gini_results = calculate_periodic_gini(df_sb_demo_scorefpd10, 'sb_demo_score', 'ln_fpd10_flag', 'FPD10')
sb_demo_scoreFPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{sb_demo_scoreFPD10.shape}")
sb_demo_scoreFPD10.columns.values

The shape of dataframe after copy is:	(108, 7)


array(['start_date', 'end_date', 'sb_demo_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [34]:
sb_demo_scoreFPD10.tail()

,start_date,end_date,sb_demo_score_FPD10_gini,period,Model_Name,version,bad_rate
103,2025-01-01,2025-01-31,0.213186,Month,sb_demo_score,1.1.0,FPD10
104,2025-01-06,2025-01-12,0.186708,Week,sb_demo_score,1.1.0,FPD10
105,2025-01-13,2025-01-19,0.211580,Week,sb_demo_score,1.1.0,FPD10
106,2025-01-20,2025-01-26,0.172097,Week,sb_demo_score,1.1.0,FPD10
107,2025-01-27,2025-02-02,0.351495,Week,sb_demo_score,1.1.0,FPD10


## FPD30

In [35]:
sq = """
with sb_demo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    beta_demo_score sb_demo_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    beta_demo_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from sb_demo_score;
"""

df_sb_demo_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID df69984e-3192-4e34-a1a3-53d7a4a09755 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [36]:
gini_results = calculate_periodic_gini(df_sb_demo_scorefpd30, 'sb_demo_score', 'ln_fpd30_flag', 'FPD30')
sb_demo_scoreFPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{sb_demo_scoreFPD30.shape}")
sb_demo_scoreFPD30.columns.values

The shape of dataframe after copy is:	(105, 7)


array(['start_date', 'end_date', 'sb_demo_score_FPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [37]:
sb_demo_scoreFPD30.head() 

,start_date,end_date,sb_demo_score_FPD30_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.289091,Week,sb_demo_score,1.1.0,FPD30
1,2023-06-01,2023-06-30,0.094755,Month,sb_demo_score,1.1.0,FPD30
2,2023-06-05,2023-06-11,0.207407,Week,sb_demo_score,1.1.0,FPD30
3,2023-06-12,2023-06-18,0.109718,Week,sb_demo_score,1.1.0,FPD30
4,2023-06-19,2023-06-25,-0.243641,Week,sb_demo_score,1.1.0,FPD30


## FSPD30

In [38]:
sq = """
with sb_demo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    beta_demo_score sb_demo_score,
    ln_fspd30_flag,   -- fspd30
	ln_mature_fspd30_flag,	--- fspd30 observation
	FROM 
   risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    beta_demo_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from sb_demo_score;
"""

df_sb_demo_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID b9e2f1a4-ee16-4a25-8693-f26f07e0211e successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [39]:
gini_results = calculate_periodic_gini(df_sb_demo_scorefspd30, 'sb_demo_score', 'ln_fspd30_flag', 'FSPD30')
sb_demo_scoreFSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{sb_demo_scoreFSPD30.shape}")
sb_demo_scoreFSPD30.columns.values

The shape of dataframe after copy is:	(100, 7)


array(['start_date', 'end_date', 'sb_demo_score_FSPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [40]:
sb_demo_scoreFSPD30.head()

,start_date,end_date,sb_demo_score_FSPD30_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.087601,Week,sb_demo_score,1.1.0,FSPD30
1,2023-06-01,2023-06-30,0.116883,Month,sb_demo_score,1.1.0,FSPD30
2,2023-06-05,2023-06-11,0.239192,Week,sb_demo_score,1.1.0,FSPD30
3,2023-06-12,2023-06-18,0.042081,Week,sb_demo_score,1.1.0,FSPD30
4,2023-06-19,2023-06-25,0.077598,Week,sb_demo_score,1.1.0,FSPD30


## FSTPD30

In [41]:
sq = """
with sb_demo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    beta_demo_score sb_demo_score,
    ln_fstpd30_flag,   -- fstpd30
	ln_mature_fstpd30_flag,	--- fstpd30 observation
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    beta_demo_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from sb_demo_score;
"""

df_sb_demo_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID d83452bb-22e9-40d7-9089-25aeeaa5da1b successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [42]:
gini_results = calculate_periodic_gini(df_sb_demo_scorefstpd30, 'sb_demo_score', 'ln_fstpd30_flag', 'FSTPD30')
sb_demo_scoreFSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{sb_demo_scoreFSTPD30.shape}")
sb_demo_scoreFSTPD30.columns.values

The shape of dataframe after copy is:	(94, 7)


array(['start_date', 'end_date', 'sb_demo_score_FSTPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [43]:
sb_demo_scoreFSTPD30.head()

,start_date,end_date,sb_demo_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.210240,Week,sb_demo_score,1.1.0,FSTPD30
1,2023-06-01,2023-06-30,0.092642,Month,sb_demo_score,1.1.0,FSTPD30
2,2023-06-05,2023-06-11,0.291754,Week,sb_demo_score,1.1.0,FSTPD30
3,2023-06-12,2023-06-18,-0.121763,Week,sb_demo_score,1.1.0,FSTPD30
4,2023-06-19,2023-06-25,0.098133,Week,sb_demo_score,1.1.0,FSTPD30


# Combining sb demo score

In [44]:
import functools

dataframes = [sb_demo_scoreFPD10, sb_demo_scoreFPD30, sb_demo_scoreFSPD30, sb_demo_scoreFSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'sb_demo_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'sb_demo_score_FPD30_gini',
       'sb_demo_score_FSPD30_gini', 'sb_demo_score_FSTPD30_gini'],
      dtype=object)

In [45]:
final_df = final_df[['start_date', 'end_date', 'period',
       'Model_Name', 'version', 'bad_rate','sb_demo_score_FPD10_gini','sb_demo_score_FPD30_gini',
       'sb_demo_score_FSPD30_gini', 'sb_demo_score_FSTPD30_gini']].copy()

## creating sb demo score table 

In [46]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_sb_demo_score;"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=c9995d8c-b0ad-4bc2-a3da-c43312de29f8>

In [47]:


import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('sb_demo_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('sb_demo_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('sb_demo_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('sb_demo_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_sb_demo_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=ca652b7f-0507-487b-a0d8-0d733640268a>

# s_cic_score

## FPD10

In [48]:
sq = """
with s_cic_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    cic_score s_cic_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    cic_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from s_cic_score;
"""

df_s_cic_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID 5412f69c-36ed-4ebc-9ccf-7a8f7da9426b successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [49]:
df_s_cic_scorefpd10.head()

,disbursementdate,digitalLoanAccountId,s_cic_score,ln_fpd10_flag,ln_mature_fpd10_flag
0,2024-07-16 19:09:27,a731f941-ed8a-4278-9ba4-1fd41132aa5e,0.117005,0,1
1,2024-09-15 09:21:50,2f8156e6-5c5b-454c-92d9-628291e6f99e,0.122660,0,1
2,2024-12-29 18:02:03,6548e088-8d8e-47ae-b33d-e6df00d80c38,0.058266,0,1
3,2025-01-14 17:34:10,15667458-5a69-445e-bd5d-4e22394b789e,0.098974,0,1
4,2025-01-11 16:07:45,350e8b35-ba30-41e6-992f-ab0a8208c404,0.098974,0,1


In [50]:
gini_results = calculate_periodic_gini(df_s_cic_scorefpd10, 's_cic_score', 'ln_fpd10_flag', 'FPD10')
s_cic_scoreFPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{s_cic_scoreFPD10.shape}")
s_cic_scoreFPD10.columns.values

The shape of dataframe after copy is:	(107, 7)


array(['start_date', 'end_date', 's_cic_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [51]:
s_cic_scoreFPD10.tail()

,start_date,end_date,s_cic_score_FPD10_gini,period,Model_Name,version,bad_rate
102,2025-01-01,2025-01-31,0.183128,Month,s_cic_score,1.1.0,FPD10
103,2025-01-06,2025-01-12,0.190109,Week,s_cic_score,1.1.0,FPD10
104,2025-01-13,2025-01-19,0.141473,Week,s_cic_score,1.1.0,FPD10
105,2025-01-20,2025-01-26,0.164915,Week,s_cic_score,1.1.0,FPD10
106,2025-01-27,2025-02-02,0.209019,Week,s_cic_score,1.1.0,FPD10


## FPD30

In [53]:
sq = """
with s_cic_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    cic_score s_cic_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
  risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    cic_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from s_cic_score;
"""

df_s_cic_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')





Job ID cf0138c4-d961-496f-ac4f-a0c97acf1e3f successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(




Query is running:   0%|          |


Downloading: 100%|██████████|


In [54]:
df_s_cic_scorefpd30.head()

,disbursementdate,digitalLoanAccountId,s_cic_score,ln_fpd30_flag,ln_mature_fpd30_flag
0,2024-07-16 19:09:27,a731f941-ed8a-4278-9ba4-1fd41132aa5e,0.117005,0,1
1,2024-09-15 09:21:50,2f8156e6-5c5b-454c-92d9-628291e6f99e,0.122660,0,1
2,2024-12-29 18:02:03,6548e088-8d8e-47ae-b33d-e6df00d80c38,0.058266,0,1
3,2024-10-30 19:26:13,4266c053-98c9-4825-bfce-b59264b1251d,0.151945,0,1
4,2024-08-18 13:17:01,d1d7594f-fcac-4295-b037-d998766d91be,0.052231,0,1


In [56]:
gini_results = calculate_periodic_gini(df_s_cic_scorefpd30, 's_cic_score', 'ln_fpd30_flag', 'FPD30')
s_cic_scoreFPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{s_cic_scoreFPD30.shape}")
s_cic_scoreFPD30.columns.values

The shape of dataframe after copy is:	(104, 7)


array(['start_date', 'end_date', 's_cic_score_FPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [57]:
s_cic_scoreFPD30.head()

,start_date,end_date,s_cic_score_FPD30_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.261538,Week,s_cic_score,1.1.0,FPD30
1,2023-06-01,2023-06-30,0.158412,Month,s_cic_score,1.1.0,FPD30
2,2023-06-05,2023-06-11,0.196970,Week,s_cic_score,1.1.0,FPD30
3,2023-06-12,2023-06-18,-0.453020,Week,s_cic_score,1.1.0,FPD30
4,2023-06-19,2023-06-25,0.435484,Week,s_cic_score,1.1.0,FPD30


In [58]:
s_cic_scoreFPD30.describe()

,start_date,end_date,s_cic_score_FPD30_gini
count,104,104,104.000000
mean,2024-03-20 01:36:55.384615424,2024-03-30 14:18:27.692307712,0.256690
min,2023-05-29 00:00:00,2023-06-04 00:00:00,-0.453020
25%,2023-10-28 06:00:00,2023-11-03 18:00:00,0.192568
50%,2024-03-21 12:00:00,2024-03-31 00:00:00,0.265936
75%,2024-08-13 18:00:00,2024-08-26 12:00:00,0.324917
max,2025-01-06 00:00:00,2025-01-31 00:00:00,0.749150
std,NaN,NaN,0.133006


## FSPD30

In [59]:
sq = """
with s_cic_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    cic_score s_cic_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    cic_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from s_cic_score;
"""

df_s_cic_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID eac82874-bc23-4096-bf4b-28c37f5a8ef1 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Query is running:   0%|          |
Downloading: 100%|██████████|


In [60]:
gini_results = calculate_periodic_gini(df_s_cic_scorefspd30, 's_cic_score', 'ln_fspd30_flag', 'FSPD30')
s_cic_scoreFSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{s_cic_scoreFSPD30.shape}")
s_cic_scoreFSPD30.columns.values

The shape of dataframe after copy is:	(99, 7)


array(['start_date', 'end_date', 's_cic_score_FSPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [61]:
s_cic_scoreFSPD30.describe()

,start_date,end_date,s_cic_score_FSPD30_gini
count,99,99,99.000000
mean,2024-03-05 19:09:05.454545408,2024-03-16 07:30:54.545454592,0.267143
min,2023-05-29 00:00:00,2023-06-04 00:00:00,0.042659
25%,2023-10-19 12:00:00,2023-10-30 00:00:00,0.214229
50%,2024-03-04 00:00:00,2024-03-17 00:00:00,0.275248
75%,2024-07-25 12:00:00,2024-08-02 00:00:00,0.313029
max,2024-12-09 00:00:00,2024-12-31 00:00:00,0.545299
std,NaN,NaN,0.082652


In [62]:
s_cic_scoreFSPD30.tail()

,start_date,end_date,s_cic_score_FSPD30_gini,period,Model_Name,version,bad_rate
94,2024-11-18,2024-11-24,0.247155,Week,s_cic_score,1.1.0,FSPD30
95,2024-11-25,2024-12-01,0.286385,Week,s_cic_score,1.1.0,FSPD30
96,2024-12-01,2024-12-31,0.297500,Month,s_cic_score,1.1.0,FSPD30
97,2024-12-02,2024-12-08,0.279570,Week,s_cic_score,1.1.0,FSPD30
98,2024-12-09,2024-12-15,0.402519,Week,s_cic_score,1.1.0,FSPD30


## FSTPD30

In [63]:
sq = """
with s_cic_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    cic_score s_cic_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    cic_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from s_cic_score;
"""

df_s_cic_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID e41c9a5c-97e2-4faa-8f47-b393d449132a successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [64]:
gini_results = calculate_periodic_gini(df_s_cic_scorefstpd30, 's_cic_score', 'ln_fstpd30_flag', 'FSTPD30')
s_cic_scoreFSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{s_cic_scoreFSTPD30.shape}")
s_cic_scoreFSTPD30.columns.values

The shape of dataframe after copy is:	(93, 7)


array(['start_date', 'end_date', 's_cic_score_FSTPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [65]:
s_cic_scoreFSTPD30.describe()

,start_date,end_date,s_cic_score_FSTPD30_gini
count,93,93,93.000000
mean,2024-02-17 16:30:58.064516096,2024-02-28 05:40:38.709677312,0.250439
min,2023-05-29 00:00:00,2023-06-04 00:00:00,-0.088419
25%,2023-10-09 00:00:00,2023-10-22 00:00:00,0.212355
50%,2024-02-19 00:00:00,2024-02-29 00:00:00,0.263496
75%,2024-07-01 00:00:00,2024-07-07 00:00:00,0.304424
max,2024-11-04 00:00:00,2024-11-30 00:00:00,0.479082
std,NaN,NaN,0.083904


In [66]:
s_cic_scoreFSTPD30.tail()

,start_date,end_date,s_cic_score_FSTPD30_gini,period,Model_Name,version,bad_rate
88,2024-10-14,2024-10-20,0.283938,Week,s_cic_score,1.1.0,FSTPD30
89,2024-10-21,2024-10-27,0.267940,Week,s_cic_score,1.1.0,FSTPD30
90,2024-10-28,2024-11-03,0.225835,Week,s_cic_score,1.1.0,FSTPD30
91,2024-11-01,2024-11-30,0.295004,Month,s_cic_score,1.1.0,FSTPD30
92,2024-11-04,2024-11-10,0.327550,Week,s_cic_score,1.1.0,FSTPD30


# Combining s_cic_score

In [67]:
import functools

dataframes = [s_cic_scoreFPD10, s_cic_scoreFPD30, s_cic_scoreFSPD30, s_cic_scoreFSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 's_cic_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 's_cic_score_FPD30_gini',
       's_cic_score_FSPD30_gini', 's_cic_score_FSTPD30_gini'],
      dtype=object)

In [68]:
final_df = final_df[['start_date', 'end_date', 'period',
       'Model_Name', 'version', 'bad_rate','s_cic_score_FPD10_gini','s_cic_score_FPD30_gini', 's_cic_score_FSPD30_gini', 's_cic_score_FSTPD30_gini']].copy()

## Creating the table

In [69]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_s_cic_score;"""

client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=c67f110c-b6ce-4605-a336-f45775c91f6a>

In [70]:


import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('s_cic_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('s_cic_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('s_cic_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('s_cic_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_s_cic_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=14e57329-4e11-4984-ab22-ebaf567f2c9a>

# sb_stack_score

## FPD10

In [71]:
# sb_stack_score

sq = """
with sb_stack_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    beta_stack_score sb_stack_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    beta_stack_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from sb_stack_score;
"""

df_sb_stack_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

df_sb_stack_scorefpd10.head()

Job ID 2e85acfc-d1bd-4ff5-97c0-cc41cfca3462 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,sb_stack_score,ln_fpd10_flag,ln_mature_fpd10_flag
0,2024-09-01 15:18:37,1a2cb6f2-e66f-444a-ba22-a05cb5652ef2,0.063497,0,1
1,2024-05-02 17:24:11,9b118965-74ea-4832-a640-8b5ea2cc140e,0.011423,0,1
2,2024-09-19 20:32:20,4f7bb0f5-ec5b-40c0-a145-6d8aa9b083ee,0.078779,1,1
3,2024-06-19 20:08:28,210afa02-21e2-4ecc-8e28-06969222c968,0.160364,0,1
4,2025-01-27 12:33:16,107d411b-cd24-4dde-86a8-6f7eeeaaa7bb,0.018078,0,1


In [72]:
gini_results = calculate_periodic_gini(df_sb_stack_scorefpd10, 'sb_stack_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

The shape of dataframe after copy is:	(108, 7)


array(['start_date', 'end_date', 'sb_stack_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [73]:
M1FPD10.describe()

,start_date,end_date,sb_stack_score_FPD10_gini
count,108,108,108.000000
mean,2024-03-26 08:13:20,2024-04-05 16:53:20,0.342652
min,2023-05-29 00:00:00,2023-06-04 00:00:00,-0.030303
25%,2023-10-28 06:00:00,2023-11-03 18:00:00,0.296225
50%,2024-03-28 12:00:00,2024-04-03 12:00:00,0.324847
75%,2024-08-27 12:00:00,2024-09-02 18:00:00,0.412945
max,2025-01-27 00:00:00,2025-02-02 00:00:00,0.589103
std,NaN,NaN,0.094668


In [74]:
M1FPD10.head()

,start_date,end_date,sb_stack_score_FPD10_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.506255,Week,sb_stack_score,1.1.0,FPD10
1,2023-06-01,2023-06-30,0.296561,Month,sb_stack_score,1.1.0,FPD10
2,2023-06-05,2023-06-11,0.407799,Week,sb_stack_score,1.1.0,FPD10
3,2023-06-12,2023-06-18,0.589103,Week,sb_stack_score,1.1.0,FPD10
4,2023-06-19,2023-06-25,-0.030303,Week,sb_stack_score,1.1.0,FPD10


## FPD30

In [75]:
# sb_stack_score

sq = """
with sb_stack_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    beta_stack_score sb_stack_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    beta_stack_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from sb_stack_score;
"""

df_sb_stack_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

df_sb_stack_scorefpd30.head()

Job ID 1be59c12-4d43-4cf1-9f61-5cec69e6f541 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,sb_stack_score,ln_fpd30_flag,ln_mature_fpd30_flag
0,2024-09-01 15:18:37,1a2cb6f2-e66f-444a-ba22-a05cb5652ef2,0.063497,0,1
1,2024-05-02 17:24:11,9b118965-74ea-4832-a640-8b5ea2cc140e,0.011423,0,1
2,2024-09-19 20:32:20,4f7bb0f5-ec5b-40c0-a145-6d8aa9b083ee,0.078779,1,1
3,2024-06-19 20:08:28,210afa02-21e2-4ecc-8e28-06969222c968,0.160364,0,1
4,2024-05-13 17:34:23,afb2601c-9ce3-4e88-96dd-38784a818625,0.015368,0,1


In [76]:
gini_results = calculate_periodic_gini(df_sb_stack_scorefpd30, 'sb_stack_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

The shape of dataframe after copy is:	(105, 7)


array(['start_date', 'end_date', 'sb_stack_score_FPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [77]:
M2FPD30.describe()

,start_date,end_date,sb_stack_score_FPD30_gini
count,105,105,105.000000
mean,2024-03-17 18:44:34.285714176,2024-03-28 06:24:00,0.361858
min,2023-05-29 00:00:00,2023-06-04 00:00:00,-0.036145
25%,2023-10-23 00:00:00,2023-10-31 00:00:00,0.314057
50%,2024-03-18 00:00:00,2024-03-31 00:00:00,0.352013
75%,2024-08-12 00:00:00,2024-08-25 00:00:00,0.423529
max,2025-01-06 00:00:00,2025-01-31 00:00:00,0.564768
std,NaN,NaN,0.094556


In [78]:
M2FPD30.tail()

,start_date,end_date,sb_stack_score_FPD30_gini,period,Model_Name,version,bad_rate
100,2024-12-16,2024-12-22,0.306499,Week,sb_stack_score,1.1.0,FPD30
101,2024-12-23,2024-12-29,0.331995,Week,sb_stack_score,1.1.0,FPD30
102,2024-12-30,2025-01-05,0.289278,Week,sb_stack_score,1.1.0,FPD30
103,2025-01-01,2025-01-31,0.276003,Month,sb_stack_score,1.1.0,FPD30
104,2025-01-06,2025-01-12,0.324793,Week,sb_stack_score,1.1.0,FPD30


## FSPD30

In [79]:
# sb_stack_score

sq = """
with sb_stack_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    beta_stack_score sb_stack_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    beta_stack_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from sb_stack_score;
"""

df_sb_stack_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

df_sb_stack_scorefspd30.head()

Job ID eee0fe71-04d8-4436-905c-0c2a8726b6a2 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,sb_stack_score,ln_fspd30_flag,ln_mature_fspd30_flag
0,2024-07-16 19:09:27,a731f941-ed8a-4278-9ba4-1fd41132aa5e,0.107962,0,1
1,2024-02-26 15:08:47,acebf8de-bd42-4b00-83bc-d7c0999a7488,0.060378,0,1
2,2024-09-15 09:21:50,2f8156e6-5c5b-454c-92d9-628291e6f99e,0.075346,0,1
3,2024-10-13 17:33:13,c58cd395-1d70-4ec0-91c5-8a067645c73c,0.044324,0,1
4,2024-07-27 19:43:54,cae2c508-fe59-47b9-8eef-fc5f6f08a184,0.055323,0,1


In [80]:
gini_results = calculate_periodic_gini(df_sb_stack_scorefspd30, 'sb_stack_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
M3FSPD30.columns.values

The shape of dataframe after copy is:	(100, 7)


array(['start_date', 'end_date', 'sb_stack_score_FSPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [81]:
M3FSPD30.describe()

,start_date,end_date,sb_stack_score_FSPD30_gini
count,100,100,100.000000
mean,2024-03-03 12:57:36,2024-03-14 00:14:24,0.365010
min,2023-05-29 00:00:00,2023-06-04 00:00:00,0.179581
25%,2023-10-14 06:00:00,2023-10-27 06:00:00,0.316516
50%,2024-03-02 12:00:00,2024-03-13 12:00:00,0.347746
75%,2024-07-23 18:00:00,2024-08-01 00:00:00,0.423752
max,2024-12-09 00:00:00,2024-12-31 00:00:00,0.551886
std,NaN,NaN,0.076612


In [82]:
M3FSPD30.tail()

,start_date,end_date,sb_stack_score_FSPD30_gini,period,Model_Name,version,bad_rate
95,2024-11-18,2024-11-24,0.314577,Week,sb_stack_score,1.1.0,FSPD30
96,2024-11-25,2024-12-01,0.300199,Week,sb_stack_score,1.1.0,FSPD30
97,2024-12-01,2024-12-31,0.321988,Month,sb_stack_score,1.1.0,FSPD30
98,2024-12-02,2024-12-08,0.317909,Week,sb_stack_score,1.1.0,FSPD30
99,2024-12-09,2024-12-15,0.327067,Week,sb_stack_score,1.1.0,FSPD30


## FSTPD30

In [83]:
# sb_stack_score

sq = """
with sb_stack_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    beta_stack_score sb_stack_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    beta_stack_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from sb_stack_score;
"""

df_sb_stack_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

df_sb_stack_scorefstpd30.head()

Job ID 3079b169-ed72-4f1f-bcf3-c32e77d60f06 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,sb_stack_score,ln_fstpd30_flag,ln_mature_fstpd30_flag
0,2024-07-16 19:09:27,a731f941-ed8a-4278-9ba4-1fd41132aa5e,0.107962,0,1
1,2024-02-26 15:08:47,acebf8de-bd42-4b00-83bc-d7c0999a7488,0.060378,0,1
2,2024-09-15 09:21:50,2f8156e6-5c5b-454c-92d9-628291e6f99e,0.075346,0,1
3,2024-10-13 17:33:13,c58cd395-1d70-4ec0-91c5-8a067645c73c,0.044324,0,1
4,2024-07-27 19:43:54,cae2c508-fe59-47b9-8eef-fc5f6f08a184,0.055323,0,1


In [84]:
gini_results = calculate_periodic_gini(df_sb_stack_scorefstpd30, 'sb_stack_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
M4FSTPD30.columns.values

The shape of dataframe after copy is:	(94, 7)


array(['start_date', 'end_date', 'sb_stack_score_FSTPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [85]:
M4FSTPD30.describe()

,start_date,end_date,sb_stack_score_FSTPD30_gini
count,94,94,94.000000
mean,2024-02-15 11:14:02.553191424,2024-02-25 23:14:02.553191424,0.339457
min,2023-05-29 00:00:00,2023-06-04 00:00:00,0.160057
25%,2023-10-03 18:00:00,2023-10-16 18:00:00,0.297767
50%,2024-02-15 12:00:00,2024-02-27 00:00:00,0.334322
75%,2024-06-29 06:00:00,2024-07-05 06:00:00,0.385845
max,2024-11-04 00:00:00,2024-11-30 00:00:00,0.500133
std,NaN,NaN,0.062929


In [86]:
M4FSTPD30.tail()

,start_date,end_date,sb_stack_score_FSTPD30_gini,period,Model_Name,version,bad_rate
89,2024-10-14,2024-10-20,0.275123,Week,sb_stack_score,1.1.0,FSTPD30
90,2024-10-21,2024-10-27,0.298238,Week,sb_stack_score,1.1.0,FSTPD30
91,2024-10-28,2024-11-03,0.287414,Week,sb_stack_score,1.1.0,FSTPD30
92,2024-11-01,2024-11-30,0.326381,Month,sb_stack_score,1.1.0,FSTPD30
93,2024-11-04,2024-11-10,0.305393,Week,sb_stack_score,1.1.0,FSTPD30


## Combining the dataframes

In [87]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'sb_stack_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'sb_stack_score_FPD30_gini',
       'sb_stack_score_FSPD30_gini', 'sb_stack_score_FSTPD30_gini'],
      dtype=object)

In [88]:
final_df = final_df[['start_date', 'end_date', 'period', 'Model_Name', 'version', 'bad_rate', 'sb_stack_score_FPD10_gini', 'sb_stack_score_FPD30_gini',  'sb_stack_score_FSPD30_gini', 'sb_stack_score_FSTPD30_gini']].copy()

## Creating the table 

In [89]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_sb_stack_score;"""

client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=4503bbff-3452-432a-898b-0dbb27a46f99>

In [90]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('sb_stack_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('sb_stack_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('sb_stack_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('sb_stack_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_sb_stack_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=452e4c50-8fbf-4aab-ac62-e99ba12b8d4a>

# sa_stack_score

## FPD10

In [91]:
# sa_stack_score

sq = """
with sa_stack_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    alpha_stack_score sa_stack_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    alpha_stack_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from sa_stack_score;
"""

sa_stack_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

sa_stack_scorefpd10.head()

Job ID 8a99fbf7-b448-40a4-a365-2cf611d25dad successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,sa_stack_score,ln_fpd10_flag,ln_mature_fpd10_flag
0,2024-07-16 19:09:27,a731f941-ed8a-4278-9ba4-1fd41132aa5e,0.069974,0,1
1,2024-09-15 09:21:50,2f8156e6-5c5b-454c-92d9-628291e6f99e,0.045933,0,1
2,2024-12-29 18:02:03,6548e088-8d8e-47ae-b33d-e6df00d80c38,0.054839,0,1
3,2025-01-14 17:34:10,15667458-5a69-445e-bd5d-4e22394b789e,0.020040,0,1
4,2025-01-11 16:07:45,350e8b35-ba30-41e6-992f-ab0a8208c404,0.025584,0,1


In [92]:
sa_stack_scorefpd10.head()

,disbursementdate,digitalLoanAccountId,sa_stack_score,ln_fpd10_flag,ln_mature_fpd10_flag
0,2024-07-16 19:09:27,a731f941-ed8a-4278-9ba4-1fd41132aa5e,0.069974,0,1
1,2024-09-15 09:21:50,2f8156e6-5c5b-454c-92d9-628291e6f99e,0.045933,0,1
2,2024-12-29 18:02:03,6548e088-8d8e-47ae-b33d-e6df00d80c38,0.054839,0,1
3,2025-01-14 17:34:10,15667458-5a69-445e-bd5d-4e22394b789e,0.020040,0,1
4,2025-01-11 16:07:45,350e8b35-ba30-41e6-992f-ab0a8208c404,0.025584,0,1


In [93]:
gini_results = calculate_periodic_gini(sa_stack_scorefpd10, 'sa_stack_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

The shape of dataframe after copy is:	(107, 7)


array(['start_date', 'end_date', 'sa_stack_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [94]:
M1FPD10.describe()

,start_date,end_date,sa_stack_score_FPD10_gini
count,107,107,107.000000
mean,2024-03-28 15:28:35.887850496,2024-04-08 01:07:17.383177472,0.378135
min,2023-05-29 00:00:00,2023-06-04 00:00:00,-0.138710
25%,2023-10-31 00:00:00,2023-11-08 12:00:00,0.334396
50%,2024-04-01 00:00:00,2024-04-07 00:00:00,0.374921
75%,2024-08-29 00:00:00,2024-09-04 12:00:00,0.453236
max,2025-01-27 00:00:00,2025-02-02 00:00:00,0.693277
std,NaN,NaN,0.121162


In [95]:
M1FPD10.head()

,start_date,end_date,sa_stack_score_FPD10_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.579487,Week,sa_stack_score,1.1.0,FPD10
1,2023-06-01,2023-06-30,0.115650,Month,sa_stack_score,1.1.0,FPD10
2,2023-06-05,2023-06-11,0.632035,Week,sa_stack_score,1.1.0,FPD10
3,2023-06-12,2023-06-18,0.013514,Week,sa_stack_score,1.1.0,FPD10
4,2023-06-19,2023-06-25,0.103226,Week,sa_stack_score,1.1.0,FPD10


## FPD30

In [97]:
# sa_stack_score

sq = """
with sa_stack_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    alpha_stack_score sa_stack_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    alpha_stack_score  is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from sa_stack_score;
"""

sa_stack_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

sa_stack_scorefpd30.head()

Job ID ee07a12a-a420-421b-85a7-c083e0e1c8d2 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,sa_stack_score,ln_fpd30_flag,ln_mature_fpd30_flag
0,2024-09-01 15:18:37,1a2cb6f2-e66f-444a-ba22-a05cb5652ef2,0.053850,0,1
1,2024-05-02 17:24:11,9b118965-74ea-4832-a640-8b5ea2cc140e,0.019328,0,1
2,2024-09-19 20:32:20,4f7bb0f5-ec5b-40c0-a145-6d8aa9b083ee,0.043131,1,1
3,2024-05-13 17:34:23,afb2601c-9ce3-4e88-96dd-38784a818625,0.014718,0,1
4,2024-08-16 11:20:46,85550e8b-8e44-4bb1-bce0-fc6dd3b495ab,0.086057,0,1


In [98]:
sa_stack_scorefpd30.head()

,disbursementdate,digitalLoanAccountId,sa_stack_score,ln_fpd30_flag,ln_mature_fpd30_flag
0,2024-09-01 15:18:37,1a2cb6f2-e66f-444a-ba22-a05cb5652ef2,0.053850,0,1
1,2024-05-02 17:24:11,9b118965-74ea-4832-a640-8b5ea2cc140e,0.019328,0,1
2,2024-09-19 20:32:20,4f7bb0f5-ec5b-40c0-a145-6d8aa9b083ee,0.043131,1,1
3,2024-05-13 17:34:23,afb2601c-9ce3-4e88-96dd-38784a818625,0.014718,0,1
4,2024-08-16 11:20:46,85550e8b-8e44-4bb1-bce0-fc6dd3b495ab,0.086057,0,1


In [99]:
gini_results = calculate_periodic_gini(sa_stack_scorefpd30, 'sa_stack_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

The shape of dataframe after copy is:	(104, 7)


array(['start_date', 'end_date', 'sa_stack_score_FPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [100]:
M2FPD30.describe()

,start_date,end_date,sa_stack_score_FPD30_gini
count,104,104,104.000000
mean,2024-03-20 01:36:55.384615424,2024-03-30 14:18:27.692307712,0.402929
min,2023-05-29 00:00:00,2023-06-04 00:00:00,-0.262626
25%,2023-10-28 06:00:00,2023-11-03 18:00:00,0.358042
50%,2024-03-21 12:00:00,2024-03-31 00:00:00,0.402276
75%,2024-08-13 18:00:00,2024-08-26 12:00:00,0.479677
max,2025-01-06 00:00:00,2025-01-31 00:00:00,0.784014
std,NaN,NaN,0.140303


In [101]:
M2FPD30.head()

,start_date,end_date,sa_stack_score_FPD30_gini,period,Model_Name,version,bad_rate
0,2023-05-29,2023-06-04,0.579487,Week,sa_stack_score,1.1.0,FPD30
1,2023-06-01,2023-06-30,0.113112,Month,sa_stack_score,1.1.0,FPD30
2,2023-06-05,2023-06-11,0.632035,Week,sa_stack_score,1.1.0,FPD30
3,2023-06-12,2023-06-18,0.000000,Week,sa_stack_score,1.1.0,FPD30
4,2023-06-19,2023-06-25,0.103226,Week,sa_stack_score,1.1.0,FPD30


## FSTPD30

In [102]:
# sa_stack_score

sq = """
with sa_stack_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    alpha_stack_score sa_stack_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    alpha_stack_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from sa_stack_score;
"""

sa_stack_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

sa_stack_scorefspd30.head()

Job ID 2e14c98c-abf3-4346-b6bd-0add8453686b successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,sa_stack_score,ln_fspd30_flag,ln_mature_fspd30_flag
0,2024-07-16 19:09:27,a731f941-ed8a-4278-9ba4-1fd41132aa5e,0.069974,0,1
1,2024-09-15 09:21:50,2f8156e6-5c5b-454c-92d9-628291e6f99e,0.045933,0,1
2,2024-10-30 19:26:13,4266c053-98c9-4825-bfce-b59264b1251d,0.036358,0,1
3,2024-08-18 13:17:01,d1d7594f-fcac-4295-b037-d998766d91be,0.016098,0,1
4,2024-07-26 11:14:26,e42d366d-9370-4761-9d94-e7cee5834849,0.056931,0,1


In [103]:
gini_results = calculate_periodic_gini(sa_stack_scorefspd30, 'sa_stack_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
M3FSPD30.columns.values

The shape of dataframe after copy is:	(99, 7)


array(['start_date', 'end_date', 'sa_stack_score_FSPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [104]:
M3FSPD30.tail()

,start_date,end_date,sa_stack_score_FSPD30_gini,period,Model_Name,version,bad_rate
94,2024-11-18,2024-11-24,0.366828,Week,sa_stack_score,1.1.0,FSPD30
95,2024-11-25,2024-12-01,0.360872,Week,sa_stack_score,1.1.0,FSPD30
96,2024-12-01,2024-12-31,0.393471,Month,sa_stack_score,1.1.0,FSPD30
97,2024-12-02,2024-12-08,0.375955,Week,sa_stack_score,1.1.0,FSPD30
98,2024-12-09,2024-12-15,0.430778,Week,sa_stack_score,1.1.0,FSPD30


## FSTPD30

In [106]:
# sa_stack_score

sq = """
with sa_stack_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    alpha_stack_score sa_stack_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-06-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    alpha_stack_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from sa_stack_score;
"""

sa_stack_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

sa_stack_scorefstpd30.head()

Job ID a4e32359-e8ac-49ba-8f6d-3443ee1d9559 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,sa_stack_score,ln_fstpd30_flag,ln_mature_fstpd30_flag
0,2024-09-01 15:18:37,1a2cb6f2-e66f-444a-ba22-a05cb5652ef2,0.053850,0,1
1,2024-05-02 17:24:11,9b118965-74ea-4832-a640-8b5ea2cc140e,0.019328,0,1
2,2024-05-13 17:34:23,afb2601c-9ce3-4e88-96dd-38784a818625,0.014718,0,1
3,2024-08-16 11:20:46,85550e8b-8e44-4bb1-bce0-fc6dd3b495ab,0.086057,0,1
4,2024-08-25 19:11:02,32a9d0b0-409b-4bd3-aae8-e705a06dfb95,0.101753,0,1


In [107]:
sa_stack_scorefstpd30.tail()

,disbursementdate,digitalLoanAccountId,sa_stack_score,ln_fstpd30_flag,ln_mature_fstpd30_flag
76781,2024-05-30 14:54:28,2d328902-dbc5-4334-90a4-856ecad08708,0.074796,1,1
76782,2024-11-08 15:47:56,bb893956-3f9b-490c-a7d1-deab934b4962,0.066828,1,1
76783,2024-07-07 16:23:28,2b7177ed-f0b4-4c09-a12e-3eee9a141cfe,0.084964,1,1
76784,2024-07-17 16:14:20,dd2db118-bb6d-4c39-8be5-e93a5ea3c9e0,0.159437,1,1
76785,2024-10-06 12:13:47,7e837c49-bfe0-4298-be76-5ac00d45187f,0.038666,1,1


In [108]:
gini_results = calculate_periodic_gini(sa_stack_scorefstpd30, 'sa_stack_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
M4FSTPD30.columns.values

The shape of dataframe after copy is:	(93, 7)


array(['start_date', 'end_date', 'sa_stack_score_FSTPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

## Combining the dataframes

In [109]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'sa_stack_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'sa_stack_score_FPD30_gini',
       'sa_stack_score_FSPD30_gini', 'sa_stack_score_FSTPD30_gini'],
      dtype=object)

In [110]:
final_df = final_df[['start_date', 'end_date', 'period', 'Model_Name', 'version', 'bad_rate', 'sa_stack_score_FPD10_gini', 'sa_stack_score_FPD30_gini',  'sa_stack_score_FSPD30_gini', 'sa_stack_score_FSTPD30_gini']].copy()

## Creating the table 

In [111]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_sa_stack_score;"""

client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=0082cfc5-1be5-42d6-9256-f05f41b83f25>

In [112]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('sa_stack_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('sa_stack_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('sa_stack_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('sa_stack_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_sa_stack_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=45eef8b8-11fb-4c21-a00c-32aa772ff297>

# gen_credo_score

## FPD10

In [113]:


sq = """
with gen_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_gen_score gen_credo_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
     risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    credo_gen_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from gen_credo_score;
"""

gen_credo_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

gen_credo_scorefpd10.head()

Job ID 169b138d-854c-4f72-85e6-68d118d33392 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,gen_credo_score,ln_fpd10_flag,ln_mature_fpd10_flag
0,2024-07-16 19:09:27,a731f941-ed8a-4278-9ba4-1fd41132aa5e,0.185556,0,1
1,2024-02-26 15:08:47,acebf8de-bd42-4b00-83bc-d7c0999a7488,0.110977,0,1
2,2023-12-03 17:15:55,8effa90f-b747-40b9-b42f-39691224f596,0.183422,1,1
3,2024-12-15 12:58:01,950353da-8475-4d52-964b-e334dcbd95dc,0.190540,0,1
4,2024-09-15 09:21:50,2f8156e6-5c5b-454c-92d9-628291e6f99e,0.134108,0,1


In [114]:
gini_results = calculate_periodic_gini(gen_credo_scorefpd10, 'gen_credo_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

The shape of dataframe after copy is:	(134, 7)


array(['start_date', 'end_date', 'gen_credo_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [115]:
M1FPD10.head()

,start_date,end_date,gen_credo_score_FPD10_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.246725,Month,gen_credo_score,1.1.0,FPD10
1,2023-01-02,2023-01-08,0.301125,Week,gen_credo_score,1.1.0,FPD10
2,2023-01-09,2023-01-15,0.273400,Week,gen_credo_score,1.1.0,FPD10
3,2023-01-16,2023-01-22,0.667532,Week,gen_credo_score,1.1.0,FPD10
4,2023-01-23,2023-01-29,0.086505,Week,gen_credo_score,1.1.0,FPD10


## FPD30

In [117]:
sq = """
with gen_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_gen_score gen_credo_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    credo_gen_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from gen_credo_score;
"""

gen_credo_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

gen_credo_scorefpd30.head()

Job ID 385adbea-9908-46ac-84ec-4965809248cf successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


,disbursementdate,digitalLoanAccountId,gen_credo_score,ln_fpd30_flag,ln_mature_fpd30_flag
0,2024-07-16 19:09:27,a731f941-ed8a-4278-9ba4-1fd41132aa5e,0.185556,0,1
1,2024-02-26 15:08:47,acebf8de-bd42-4b00-83bc-d7c0999a7488,0.110977,0,1
2,2023-12-03 17:15:55,8effa90f-b747-40b9-b42f-39691224f596,0.183422,1,1
3,2024-12-15 12:58:01,950353da-8475-4d52-964b-e334dcbd95dc,0.190540,0,1
4,2024-09-15 09:21:50,2f8156e6-5c5b-454c-92d9-628291e6f99e,0.134108,0,1


In [118]:
gen_credo_scorefpd30.head()

,disbursementdate,digitalLoanAccountId,gen_credo_score,ln_fpd30_flag,ln_mature_fpd30_flag
0,2024-07-16 19:09:27,a731f941-ed8a-4278-9ba4-1fd41132aa5e,0.185556,0,1
1,2024-02-26 15:08:47,acebf8de-bd42-4b00-83bc-d7c0999a7488,0.110977,0,1
2,2023-12-03 17:15:55,8effa90f-b747-40b9-b42f-39691224f596,0.183422,1,1
3,2024-12-15 12:58:01,950353da-8475-4d52-964b-e334dcbd95dc,0.190540,0,1
4,2024-09-15 09:21:50,2f8156e6-5c5b-454c-92d9-628291e6f99e,0.134108,0,1


In [119]:
gini_results = calculate_periodic_gini(gen_credo_scorefpd30, 'gen_credo_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

The shape of dataframe after copy is:	(131, 7)


array(['start_date', 'end_date', 'gen_credo_score_FPD30_gini', 'period',
       'Model_Name', 'version', 'bad_rate'], dtype=object)

In [120]:
M2FPD30.head()

,start_date,end_date,gen_credo_score_FPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.253005,Month,gen_credo_score,1.1.0,FPD30
1,2023-01-02,2023-01-08,0.342149,Week,gen_credo_score,1.1.0,FPD30
2,2023-01-09,2023-01-15,0.407163,Week,gen_credo_score,1.1.0,FPD30
3,2023-01-16,2023-01-22,0.667532,Week,gen_credo_score,1.1.0,FPD30
4,2023-01-23,2023-01-29,0.025547,Week,gen_credo_score,1.1.0,FPD30


## FSPD30

In [121]:
sq = """
with gen_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_gen_score gen_credo_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    credo_gen_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from gen_credo_score;
"""

gen_credo_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(gen_credo_scorefspd30.head(2))

gini_results = calculate_periodic_gini(gen_credo_scorefspd30, 'gen_credo_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
print(M3FSPD30.columns.values)

M3FSPD30.head()

Job ID d6259926-d684-494f-b5c6-cc847b4af659 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  gen_credo_score  \
0 2024-09-01 15:18:37  1a2cb6f2-e66f-444a-ba22-a05cb5652ef2         0.113903   
1 2024-05-02 17:24:11  9b118965-74ea-4832-a640-8b5ea2cc140e         0.097561   

   ln_fspd30_flag  ln_mature_fspd30_flag  
0               0                      1  
1               0                      1  
The shape of dataframe after copy is:	(126, 7)
['start_date' 'end_date' 'gen_credo_score_FSPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,gen_credo_score_FSPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.172055,Month,gen_credo_score,1.1.0,FSPD30
1,2023-01-02,2023-01-08,0.262803,Week,gen_credo_score,1.1.0,FSPD30
2,2023-01-09,2023-01-15,0.189744,Week,gen_credo_score,1.1.0,FSPD30
3,2023-01-16,2023-01-22,0.410738,Week,gen_credo_score,1.1.0,FSPD30
4,2023-01-23,2023-01-29,0.073579,Week,gen_credo_score,1.1.0,FSPD30


## FSTPD30

In [122]:
sq = """
with gen_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_gen_score gen_credo_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    credo_gen_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from gen_credo_score;
"""

gen_credo_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(gen_credo_scorefstpd30.head(2))

gini_results = calculate_periodic_gini(gen_credo_scorefstpd30, 'gen_credo_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
print(M4FSTPD30.columns.values)

M4FSTPD30.head()

Job ID 219acb2e-afe7-48cb-8dab-b995a000f6c3 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  gen_credo_score  \
0 2024-09-01 15:18:37  1a2cb6f2-e66f-444a-ba22-a05cb5652ef2         0.113903   
1 2024-05-02 17:24:11  9b118965-74ea-4832-a640-8b5ea2cc140e         0.097561   

   ln_fstpd30_flag  ln_mature_fstpd30_flag  
0                0                       1  
1                0                       1  
The shape of dataframe after copy is:	(120, 7)
['start_date' 'end_date' 'gen_credo_score_FSTPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,gen_credo_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.180837,Month,gen_credo_score,1.1.0,FSTPD30
1,2023-01-02,2023-01-08,0.282959,Week,gen_credo_score,1.1.0,FSTPD30
2,2023-01-09,2023-01-15,0.189908,Week,gen_credo_score,1.1.0,FSTPD30
3,2023-01-16,2023-01-22,0.198840,Week,gen_credo_score,1.1.0,FSTPD30
4,2023-01-23,2023-01-29,0.162845,Week,gen_credo_score,1.1.0,FSTPD30


## Combining tables 

In [123]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'gen_credo_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'gen_credo_score_FPD30_gini',
       'gen_credo_score_FSPD30_gini', 'gen_credo_score_FSTPD30_gini'],
      dtype=object)

In [124]:
final_df = final_df[['start_date', 'end_date','period',
       'Model_Name', 'version', 'bad_rate', 'gen_credo_score_FPD10_gini','gen_credo_score_FPD30_gini',   'gen_credo_score_FSPD30_gini', 'gen_credo_score_FSTPD30_gini']].copy()

In [125]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_gen_credo_score;"""

client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=a0b3124d-0bfa-47e1-927b-7728b02ff075>

In [126]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('gen_credo_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('gen_credo_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('gen_credo_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('gen_credo_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_gen_credo_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=0059e4d9-4127-440c-98e0-f166c1d0144e>

# c_credo_score

In [128]:
sq = """
with c_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_cash_score c_credo_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    credo_cash_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from c_credo_score;
"""

c_credo_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

c_credo_scorefpd10.head()

gini_results = calculate_periodic_gini(c_credo_scorefpd10, 'c_credo_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

# FPD30

sq = """
with c_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_cash_score c_credo_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    credo_cash_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from c_credo_score;
"""

c_credo_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

c_credo_scorefpd30.head()

gini_results = calculate_periodic_gini(c_credo_scorefpd30, 'c_credo_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

sq = """
with c_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_cash_score c_credo_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
   risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    credo_cash_score  is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from c_credo_score;
"""

c_credo_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(c_credo_scorefspd30.head(2))

gini_results = calculate_periodic_gini(c_credo_scorefspd30, 'c_credo_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
print(M3FSPD30.columns.values)

M3FSPD30.head()

sq = """
with c_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_cash_score c_credo_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    credo_cash_score  is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from c_credo_score;
"""

c_credo_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(c_credo_scorefstpd30.head(2))

gini_results = calculate_periodic_gini(c_credo_scorefstpd30, 'c_credo_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
print(M4FSTPD30.columns.values)

M4FSTPD30.head()







Job ID 91a487c1-c4fa-4990-bef4-b2323bba0b59 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(




Query is running:   0%|          |



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(134, 7)
Job ID 6ad7ecd6-af2c-4261-b2d7-e087573d08af successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(131, 7)
Job ID 1d5b7832-d031-4339-9447-b8496b7109a4 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  c_credo_score  \
0 2024-09-01 15:18:37  1a2cb6f2-e66f-444a-ba22-a05cb5652ef2       0.130117   
1 2024-05-02 17:24:11  9b118965-74ea-4832-a640-8b5ea2cc140e       0.075873   

   ln_fspd30_flag  ln_mature_fspd30_flag  
0               0                      1  
1               0                      1  
The shape of dataframe after copy is:	(126, 7)
['start_date' 'end_date' 'c_credo_score_FSPD30_gini' 'period' 'Model_Name'
 'version' 'bad_rate']
Job ID 74de92d9-a527-4d94-b0e7-5d9598e21414 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  c_credo_score  \
0 2024-09-01 15:18:37  1a2cb6f2-e66f-444a-ba22-a05cb5652ef2       0.130117   
1 2024-05-02 17:24:11  9b118965-74ea-4832-a640-8b5ea2cc140e       0.075873   

   ln_fstpd30_flag  ln_mature_fstpd30_flag  
0                0                       1  
1                0                       1  
The shape of dataframe after copy is:	(120, 7)
['start_date' 'end_date' 'c_credo_score_FSTPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,c_credo_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.035948,Month,c_credo_score,1.1.0,FSTPD30
1,2023-01-02,2023-01-08,0.036009,Week,c_credo_score,1.1.0,FSTPD30
2,2023-01-09,2023-01-15,0.062143,Week,c_credo_score,1.1.0,FSTPD30
3,2023-01-16,2023-01-22,0.275891,Week,c_credo_score,1.1.0,FSTPD30
4,2023-01-23,2023-01-29,-0.050558,Week,c_credo_score,1.1.0,FSTPD30


In [129]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'c_credo_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'c_credo_score_FPD30_gini',
       'c_credo_score_FSPD30_gini', 'c_credo_score_FSTPD30_gini'],
      dtype=object)

In [130]:
final_df = final_df[['start_date', 'end_date','period',    'Model_Name', 'version', 'bad_rate','c_credo_score_FPD10_gini', 'c_credo_score_FPD30_gini',    'c_credo_score_FSPD30_gini', 'c_credo_score_FSTPD30_gini']].copy()

In [131]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_c_credo_score;"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=8474f475-e0fa-4b83-9a43-22af262c26f0>

In [132]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('c_credo_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('c_credo_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('c_credo_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('c_credo_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_c_credo_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=706e36fe-3cf4-465a-a51d-1be76c796572>

# s_credo_score

In [133]:


sq = """
with s_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_sil_score s_credo_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    credo_sil_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from s_credo_score;
"""

s_credo_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

s_credo_scorefpd10.head()

gini_results = calculate_periodic_gini(s_credo_scorefpd10, 's_credo_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

# FPD30

sq = """
with s_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_sil_score s_credo_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
   risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    credo_sil_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from s_credo_score;
"""

s_credo_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

s_credo_scorefpd30.head()

gini_results = calculate_periodic_gini(s_credo_scorefpd30, 's_credo_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

sq = """
with s_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_sil_score s_credo_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    credo_sil_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from s_credo_score;
"""

s_credo_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(s_credo_scorefspd30.head(2))

gini_results = calculate_periodic_gini(s_credo_scorefspd30, 's_credo_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
print(M3FSPD30.columns.values)

M3FSPD30.head()

sq = """
with s_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_sil_score s_credo_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    prj-prod-dataplatform.risk_mart.sil_risk_ds_master_20230101_20250206
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    credo_sil_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from s_credo_score;
"""

s_credo_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(s_credo_scorefstpd30.head(2))

gini_results = calculate_periodic_gini(s_credo_scorefstpd30, 's_credo_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
print(M4FSTPD30.columns.values)

M4FSTPD30.head()



Job ID f32953d1-3d2c-4d8a-aabb-54619acecd81 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(134, 7)
Job ID 319a86d8-f3ea-446d-bcc6-de2099b944cf successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(131, 7)
Job ID ceab98f7-01c9-42da-93f2-a46103b379d3 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  s_credo_score  \
0 2023-01-09 20:02:29  851fb2f2-3a0b-4abe-a89b-6f34e183ea08       0.112890   
1 2023-03-04 11:59:14  5d0bcd27-ceed-4309-8cf5-9fac9dfc2676       0.114866   

   ln_fspd30_flag  ln_mature_fspd30_flag  
0               0                      1  
1               0                      1  
The shape of dataframe after copy is:	(123, 7)
['start_date' 'end_date' 's_credo_score_FSPD30_gini' 'period' 'Model_Name'
 'version' 'bad_rate']
Job ID 81829ade-7425-4f47-99c5-843fcc99415e successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  s_credo_score  \
0 2023-01-09 20:02:29  851fb2f2-3a0b-4abe-a89b-6f34e183ea08       0.112890   
1 2023-03-04 11:59:14  5d0bcd27-ceed-4309-8cf5-9fac9dfc2676       0.114866   

   ln_fstpd30_flag  ln_mature_fstpd30_flag  
0                0                       1  
1                0                       1  
The shape of dataframe after copy is:	(117, 7)
['start_date' 'end_date' 's_credo_score_FSTPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,s_credo_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.219086,Month,s_credo_score,1.1.0,FSTPD30
1,2023-01-02,2023-01-08,0.261708,Week,s_credo_score,1.1.0,FSTPD30
2,2023-01-09,2023-01-15,0.251056,Week,s_credo_score,1.1.0,FSTPD30
3,2023-01-16,2023-01-22,0.275062,Week,s_credo_score,1.1.0,FSTPD30
4,2023-01-23,2023-01-29,0.157554,Week,s_credo_score,1.1.0,FSTPD30


## Combining data

In [134]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 's_credo_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 's_credo_score_FPD30_gini',
       's_credo_score_FSPD30_gini', 's_credo_score_FSTPD30_gini'],
      dtype=object)

In [135]:
final_df = final_df[['start_date', 'end_date','period',    'Model_Name', 'version', 'bad_rate','s_credo_score_FPD10_gini', 's_credo_score_FPD30_gini',    's_credo_score_FSPD30_gini', 's_credo_score_FSTPD30_gini']].copy()

In [136]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_s_credo_score;"""

client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=b2420e0d-d6a0-4600-884a-2dead90d284c>

In [137]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('s_credo_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('s_credo_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('s_credo_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('s_credo_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_s_credo_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=6add640e-ea31-4b1a-881a-4c6ac88cb877>

In [138]:
final_df.head()

,start_date,end_date,period,Model_Name,version,bad_rate,s_credo_score_FPD10_gini,s_credo_score_FPD30_gini,s_credo_score_FSPD30_gini,s_credo_score_FSTPD30_gini
0,2023-01-01,2023-01-31,Month,s_credo_score,1.1.0,FPD10,0.230181,NaN,NaN,NaN
1,2023-01-02,2023-01-08,Week,s_credo_score,1.1.0,FPD10,0.248978,NaN,NaN,NaN
2,2023-01-09,2023-01-15,Week,s_credo_score,1.1.0,FPD10,0.296337,NaN,NaN,NaN
3,2023-01-16,2023-01-22,Week,s_credo_score,1.1.0,FPD10,0.529870,NaN,NaN,NaN
4,2023-01-23,2023-01-29,Week,s_credo_score,1.1.0,FPD10,0.079585,NaN,NaN,NaN


# fu_credo_score

In [139]:


sq = """
with fu_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_flexup_score fu_credo_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    credo_flexup_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from fu_credo_score;
"""

fu_credo_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

fu_credo_scorefpd10.head()

gini_results = calculate_periodic_gini(fu_credo_scorefpd10, 'fu_credo_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

# FPD30

sq = """
with fu_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_flexup_score fu_credo_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    credo_flexup_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from fu_credo_score;
"""

fu_credo_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

fu_credo_scorefpd30.head()

gini_results = calculate_periodic_gini(fu_credo_scorefpd30, 'fu_credo_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

sq = """
with fu_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_flexup_score fu_credo_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    credo_flexup_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from fu_credo_score;
"""

fu_credo_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(fu_credo_scorefspd30.head(2))

gini_results = calculate_periodic_gini(fu_credo_scorefspd30, 'fu_credo_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
print(M3FSPD30.columns.values)

M3FSPD30.head()

sq = """
with fu_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_flexup_score fu_credo_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    credo_flexup_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from fu_credo_score;
"""

fu_credo_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(fu_credo_scorefstpd30.head(2))

gini_results = calculate_periodic_gini(fu_credo_scorefstpd30, 'fu_credo_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
print(M4FSTPD30.columns.values)

M4FSTPD30.head()



Job ID 0db7a82c-0e99-422f-89bf-d86d029bdeb3 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(134, 7)
Job ID 3bea8425-7a18-4598-a86c-55fcb3e102ba successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(131, 7)
Job ID 69acd435-75bd-43fb-8d80-afc8ef393dca successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  fu_credo_score  \
0 2024-07-16 19:09:27  a731f941-ed8a-4278-9ba4-1fd41132aa5e        0.057160   
1 2024-02-26 15:08:47  acebf8de-bd42-4b00-83bc-d7c0999a7488        0.175585   

   ln_fspd30_flag  ln_mature_fspd30_flag  
0               0                      1  
1               0                      1  
The shape of dataframe after copy is:	(126, 7)
['start_date' 'end_date' 'fu_credo_score_FSPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']
Job ID f05acb2d-c3a3-4fc4-a8d1-dde576f82cde successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  fu_credo_score  \
0 2024-07-16 19:09:27  a731f941-ed8a-4278-9ba4-1fd41132aa5e        0.057160   
1 2024-02-26 15:08:47  acebf8de-bd42-4b00-83bc-d7c0999a7488        0.175585   

   ln_fstpd30_flag  ln_mature_fstpd30_flag  
0                0                       1  
1                0                       1  
The shape of dataframe after copy is:	(120, 7)
['start_date' 'end_date' 'fu_credo_score_FSTPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,fu_credo_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.072860,Month,fu_credo_score,1.1.0,FSTPD30
1,2023-01-02,2023-01-08,0.002165,Week,fu_credo_score,1.1.0,FSTPD30
2,2023-01-09,2023-01-15,0.056674,Week,fu_credo_score,1.1.0,FSTPD30
3,2023-01-16,2023-01-22,0.326429,Week,fu_credo_score,1.1.0,FSTPD30
4,2023-01-23,2023-01-29,0.109935,Week,fu_credo_score,1.1.0,FSTPD30


## Combining data

In [140]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'fu_credo_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'fu_credo_score_FPD30_gini',
       'fu_credo_score_FSPD30_gini', 'fu_credo_score_FSTPD30_gini'],
      dtype=object)

In [141]:
final_df = final_df[['start_date', 'end_date','period',    'Model_Name', 'version', 'bad_rate','fu_credo_score_FPD10_gini', 'fu_credo_score_FPD30_gini',    'fu_credo_score_FSPD30_gini', 'fu_credo_score_FSTPD30_gini']].copy()

In [142]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_fu_credo_score"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=90d69bd2-6f1e-4116-b3cb-1089e6d68db4>

In [143]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('fu_credo_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('fu_credo_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('fu_credo_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('fu_credo_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_fu_credo_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=a2bdc0fa-88ba-4926-b144-500c412369e2>

In [144]:
final_df.head()

,start_date,end_date,period,Model_Name,version,bad_rate,fu_credo_score_FPD10_gini,fu_credo_score_FPD30_gini,fu_credo_score_FSPD30_gini,fu_credo_score_FSTPD30_gini
0,2023-01-01,2023-01-31,Month,fu_credo_score,1.1.0,FPD10,0.023608,NaN,NaN,NaN
1,2023-01-02,2023-01-08,Week,fu_credo_score,1.1.0,FPD10,-0.032464,NaN,NaN,NaN
2,2023-01-09,2023-01-15,Week,fu_credo_score,1.1.0,FPD10,0.016278,NaN,NaN,NaN
3,2023-01-16,2023-01-22,Week,fu_credo_score,1.1.0,FPD10,0.153247,NaN,NaN,NaN
4,2023-01-23,2023-01-29,Week,fu_credo_score,1.1.0,FPD10,0.120242,NaN,NaN,NaN


# r_credo_score

In [145]:


sq = """
with r_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_reloan_score r_credo_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    credo_reloan_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from r_credo_score;
"""

r_credo_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

r_credo_scorefpd10.head()

gini_results = calculate_periodic_gini(r_credo_scorefpd10, 'r_credo_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

# FPD30

sq = """
with r_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_reloan_score r_credo_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    credo_reloan_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from r_credo_score;
"""

r_credo_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

r_credo_scorefpd30.head()

gini_results = calculate_periodic_gini(r_credo_scorefpd30, 'r_credo_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

sq = """
with r_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_reloan_score r_credo_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    credo_reloan_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from r_credo_score;
"""

r_credo_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(r_credo_scorefspd30.head(2))

gini_results = calculate_periodic_gini(r_credo_scorefspd30, 'r_credo_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
print(M3FSPD30.columns.values)

M3FSPD30.head()

sq = """
with r_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_reloan_score r_credo_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    credo_reloan_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from r_credo_score;
"""

r_credo_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(r_credo_scorefstpd30.head(2))

gini_results = calculate_periodic_gini(r_credo_scorefstpd30, 'r_credo_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
print(M4FSTPD30.columns.values)

M4FSTPD30.head()



Job ID bd457e38-7963-4ead-9f42-030ed8a4076e successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(134, 7)
Job ID 34406b89-529d-4b50-af75-a30d782fb39c successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(131, 7)
Job ID 338d48c4-e2ba-47f5-b386-772bbfa2bff7 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  r_credo_score  \
0 2024-07-16 19:09:27  a731f941-ed8a-4278-9ba4-1fd41132aa5e       0.366342   
1 2024-02-26 15:08:47  acebf8de-bd42-4b00-83bc-d7c0999a7488       0.308916   

   ln_fspd30_flag  ln_mature_fspd30_flag  
0               0                      1  
1               0                      1  
The shape of dataframe after copy is:	(126, 7)
['start_date' 'end_date' 'r_credo_score_FSPD30_gini' 'period' 'Model_Name'
 'version' 'bad_rate']
Job ID 94e05c7a-e698-42a3-a28f-6492c89c9f9c successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  r_credo_score  \
0 2024-09-01 15:18:37  1a2cb6f2-e66f-444a-ba22-a05cb5652ef2       0.277636   
1 2024-05-02 17:24:11  9b118965-74ea-4832-a640-8b5ea2cc140e       0.029013   

   ln_fstpd30_flag  ln_mature_fstpd30_flag  
0                0                       1  
1                0                       1  
The shape of dataframe after copy is:	(120, 7)
['start_date' 'end_date' 'r_credo_score_FSTPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,r_credo_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.048984,Month,r_credo_score,1.1.0,FSTPD30
1,2023-01-02,2023-01-08,0.014955,Week,r_credo_score,1.1.0,FSTPD30
2,2023-01-09,2023-01-15,0.193885,Week,r_credo_score,1.1.0,FSTPD30
3,2023-01-16,2023-01-22,-0.019056,Week,r_credo_score,1.1.0,FSTPD30
4,2023-01-23,2023-01-29,0.024691,Week,r_credo_score,1.1.0,FSTPD30


## Combining data

In [146]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'r_credo_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'r_credo_score_FPD30_gini',
       'r_credo_score_FSPD30_gini', 'r_credo_score_FSTPD30_gini'],
      dtype=object)

In [147]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'bad_rate','r_credo_score_FPD10_gini','r_credo_score_FPD30_gini',    'r_credo_score_FSPD30_gini', 'r_credo_score_FSTPD30_gini']].copy()

In [148]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_r_credo_score;"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=529bd70a-d981-4352-8758-88dc4c501814>

In [149]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('r_credo_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('r_credo_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('r_credo_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('r_credo_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_r_credo_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=2811a005-7b19-4c4c-bf19-300d55bef224>

# old_gen_credo_score

In [150]:


sq = """
with old_gen_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_old_gen_score old_gen_credo_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    credo_old_gen_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from old_gen_credo_score;
"""

old_gen_credo_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

old_gen_credo_scorefpd10.head()

gini_results = calculate_periodic_gini_threedigit(old_gen_credo_scorefpd10, 'old_gen_credo_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

# FPD30

sq = """
with old_gen_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_old_gen_score old_gen_credo_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    credo_old_gen_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from old_gen_credo_score;
"""

old_gen_credo_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

old_gen_credo_scorefpd30.head()

gini_results = calculate_periodic_gini_threedigit(old_gen_credo_scorefpd30, 'old_gen_credo_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

sq = """
with old_gen_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_old_gen_score old_gen_credo_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    credo_old_gen_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from old_gen_credo_score;
"""

old_gen_credo_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(old_gen_credo_scorefspd30.head(2))

gini_results = calculate_periodic_gini_threedigit(old_gen_credo_scorefspd30, 'old_gen_credo_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
print(M3FSPD30.columns.values)

M3FSPD30.head()

sq = """
with old_gen_credo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    credo_old_gen_score old_gen_credo_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    credo_old_gen_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from old_gen_credo_score;
"""

old_gen_credo_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(old_gen_credo_scorefstpd30.head(2))

gini_results = calculate_periodic_gini_threedigit(old_gen_credo_scorefstpd30, 'old_gen_credo_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
print(M4FSTPD30.columns.values)

M4FSTPD30.head()



Job ID be9bb607-f487-4f93-bf10-98d1facac7ad successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(134, 7)
Job ID be28f3c8-aa4b-4fb9-af2a-0ed7dcad6b0a successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(131, 7)
Job ID 8a5c15bc-cd55-4adf-be8c-802f9cb5570e successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  \
0 2024-05-02 17:24:11  9b118965-74ea-4832-a640-8b5ea2cc140e   
1 2023-02-02 17:24:09  538a175e-1dcc-4cd9-8527-c1b0b6bf2ef3   

  old_gen_credo_score  ln_fspd30_flag  ln_mature_fspd30_flag  
0                 507               0                      1  
1               496.0               0                      1  
The shape of dataframe after copy is:	(126, 7)
['start_date' 'end_date' 'old_gen_credo_score_FSPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']
Job ID 222dd103-5571-4086-b20e-e44877cbd756 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  \
0 2024-05-02 17:24:11  9b118965-74ea-4832-a640-8b5ea2cc140e   
1 2023-02-02 17:24:09  538a175e-1dcc-4cd9-8527-c1b0b6bf2ef3   

  old_gen_credo_score  ln_fstpd30_flag  ln_mature_fstpd30_flag  
0                 507                0                       1  
1               496.0                0                       1  
The shape of dataframe after copy is:	(120, 7)
['start_date' 'end_date' 'old_gen_credo_score_FSTPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,old_gen_credo_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.121455,Month,old_gen_credo_score,1.1.0,FSTPD30
1,2023-01-02,2023-01-08,0.229814,Week,old_gen_credo_score,1.1.0,FSTPD30
2,2023-01-09,2023-01-15,0.145360,Week,old_gen_credo_score,1.1.0,FSTPD30
3,2023-01-16,2023-01-22,0.130184,Week,old_gen_credo_score,1.1.0,FSTPD30
4,2023-01-23,2023-01-29,-0.004464,Week,old_gen_credo_score,1.1.0,FSTPD30


## Combining data

In [151]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'old_gen_credo_score_FPD10_gini',
       'period', 'Model_Name', 'version', 'bad_rate',
       'old_gen_credo_score_FPD30_gini',
       'old_gen_credo_score_FSPD30_gini',
       'old_gen_credo_score_FSTPD30_gini'], dtype=object)

In [152]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'bad_rate','old_gen_credo_score_FPD10_gini','old_gen_credo_score_FPD30_gini',    'old_gen_credo_score_FSPD30_gini', 'old_gen_credo_score_FSTPD30_gini']].copy()

In [153]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_old_gen_credo_score;"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=3ef75cfc-ac2f-41ca-823a-3716da819459>

In [154]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('old_gen_credo_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('old_gen_credo_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('old_gen_credo_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('old_gen_credo_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_old_gen_credo_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=49164b47-8ade-478a-b90e-2294be858fea>

In [155]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_old_gen_credo_score;"""

df = client.query(sq).to_dataframe(progress_bar_type='tqdm')

Job ID e9a10a97-2751-4980-b718-212d22d28b28 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|


In [156]:
df['bad_rate'].value_counts()

bad_rate
FPD10      134
FPD30      131
FSPD30     126
FSTPD30    120
Name: count, dtype: int64

# old_cic_score

In [157]:


sq = """
with old_cic_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_cic_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    old_cic_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from old_cic_score;
"""

old_cic_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

old_cic_scorefpd10.head()

gini_results = calculate_periodic_hybrid_gini(old_cic_scorefpd10, 'old_cic_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

# FPD30

sq = """
with old_cic_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_cic_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    old_cic_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from old_cic_score;
"""

old_cic_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

old_cic_scorefpd30.head()

gini_results = calculate_periodic_hybrid_gini(old_cic_scorefpd30, 'old_cic_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

sq = """
with old_cic_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_cic_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    old_cic_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from old_cic_score;
"""

old_cic_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(old_cic_scorefspd30.head(2))

gini_results = calculate_periodic_hybrid_gini(old_cic_scorefspd30, 'old_cic_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
print(M3FSPD30.columns.values)

M3FSPD30.head()

sq = """
with old_cic_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_cic_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    old_cic_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from old_cic_score;
"""

old_cic_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(old_cic_scorefstpd30.head(2))

gini_results = calculate_periodic_hybrid_gini(old_cic_scorefstpd30, 'old_cic_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
print(M4FSTPD30.columns.values)

M4FSTPD30.head()



Job ID e0fc33d1-593c-47d3-be6e-4b9d07b27edf successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(94, 7)
Job ID 3f4362c3-79a6-4008-b05f-90fce3b37876 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(91, 7)
Job ID 04e748fd-6c47-44fd-bd28-205b459d435a successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId old_cic_score  \
0 2024-08-15 19:03:54  0bc974ab-f356-4897-b8c9-1de7aa30f44c       593.000   
1 2024-03-24 16:15:13  36a51f56-9627-4685-bc0f-c961de6f6d3f       544.000   

   ln_fspd30_flag  ln_mature_fspd30_flag  
0               1                      1  
1               1                      1  
The shape of dataframe after copy is:	(86, 7)
['start_date' 'end_date' 'old_cic_score_FSPD30_gini' 'period' 'Model_Name'
 'version' 'bad_rate']
Job ID d7408c53-1992-4542-a093-17ec81d7ebc4 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId old_cic_score  \
0 2024-08-15 19:03:54  0bc974ab-f356-4897-b8c9-1de7aa30f44c       593.000   
1 2024-03-24 16:15:13  36a51f56-9627-4685-bc0f-c961de6f6d3f       544.000   

   ln_fstpd30_flag  ln_mature_fstpd30_flag  
0                1                       1  
1                1                       1  
The shape of dataframe after copy is:	(80, 7)
['start_date' 'end_date' 'old_cic_score_FSTPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,old_cic_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-08-01,2023-08-31,0.087891,Month,old_cic_score,1.1.0,FSTPD30
1,2023-08-21,2023-08-27,0.020000,Week,old_cic_score,1.1.0,FSTPD30
2,2023-08-28,2023-09-03,0.149206,Week,old_cic_score,1.1.0,FSTPD30
3,2023-09-01,2023-09-30,0.153262,Month,old_cic_score,1.1.0,FSTPD30
4,2023-09-04,2023-09-10,0.171044,Week,old_cic_score,1.1.0,FSTPD30


## Combining data

In [158]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'old_cic_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'old_cic_score_FPD30_gini',
       'old_cic_score_FSPD30_gini', 'old_cic_score_FSTPD30_gini'],
      dtype=object)

In [159]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'bad_rate','old_cic_score_FPD10_gini','old_cic_score_FPD30_gini',    'old_cic_score_FSPD30_gini', 'old_cic_score_FSTPD30_gini']].copy()

In [160]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_old_cic_score;"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=bf8c0236-de30-4bf0-9a41-c4b133211b7f>

In [161]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('old_cic_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('old_cic_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('old_cic_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('old_cic_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_old_cic_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=a469f283-2d66-4c4b-8417-88fb1d09101a>

# old_demo_score

In [162]:


sq = """
with old_demo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_demo_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    old_demo_score is not null
  AND
    ln_mature_fpd10_flag = 1
)
select * from old_demo_score;
"""

old_demo_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

old_demo_scorefpd10.head()

gini_results = calculate_periodic_hybrid_gini(old_demo_scorefpd10, 'old_demo_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

# FPD30

sq = """
with old_demo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_demo_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    old_demo_score is not null
  AND
    ln_mature_fpd30_flag = 1
)
select * from old_demo_score;
"""

old_demo_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

old_demo_scorefpd30.head()

gini_results = calculate_periodic_hybrid_gini(old_demo_scorefpd30, 'old_demo_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

sq = """
with old_demo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_demo_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    old_demo_score is not null
  AND
    ln_mature_fspd30_flag = 1
)
select * from old_demo_score;
"""

old_demo_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(old_demo_scorefspd30.head(2))

gini_results = calculate_periodic_hybrid_gini(old_demo_scorefspd30, 'old_demo_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
print(M3FSPD30.columns.values)

M3FSPD30.head()

sq = """
with old_demo_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    old_demo_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    old_demo_score is not null
  AND
    ln_mature_fstpd30_flag = 1
)
select * from old_demo_score;
"""

old_demo_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(old_demo_scorefstpd30.head(2))

gini_results = calculate_periodic_hybrid_gini(old_demo_scorefstpd30, 'old_demo_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
print(M4FSTPD30.columns.values)

M4FSTPD30.head()



Job ID 09c6b00d-3ac1-4463-ad87-3e6425fa6e9a successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(134, 7)
Job ID 21bf13f4-6c0d-4310-813c-05cb5419fa6a successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(131, 7)
Job ID b825d07a-1d95-4288-8e8d-52860e0dc754 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  old_demo_score  \
0 2023-04-25 18:08:03  ee89ea72-76ab-4e1b-a0eb-3e959907db2b           405.0   
1 2024-07-26 11:14:26  e42d366d-9370-4761-9d94-e7cee5834849           474.0   

   ln_fspd30_flag  ln_mature_fspd30_flag  
0               0                      1  
1               0                      1  
The shape of dataframe after copy is:	(126, 7)
['start_date' 'end_date' 'old_demo_score_FSPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']
Job ID c315d380-cfa6-4906-8880-9788b089e650 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  old_demo_score  \
0 2024-09-01 15:18:37  1a2cb6f2-e66f-444a-ba22-a05cb5652ef2           442.0   
1 2023-02-02 17:24:09  538a175e-1dcc-4cd9-8527-c1b0b6bf2ef3           414.0   

   ln_fstpd30_flag  ln_mature_fstpd30_flag  
0                0                       1  
1                0                       1  
The shape of dataframe after copy is:	(120, 7)
['start_date' 'end_date' 'old_demo_score_FSTPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,old_demo_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.123537,Month,old_demo_score,1.1.0,FSTPD30
1,2023-01-02,2023-01-08,0.173947,Week,old_demo_score,1.1.0,FSTPD30
2,2023-01-09,2023-01-15,0.046980,Week,old_demo_score,1.1.0,FSTPD30
3,2023-01-16,2023-01-22,0.136703,Week,old_demo_score,1.1.0,FSTPD30
4,2023-01-23,2023-01-29,0.118754,Week,old_demo_score,1.1.0,FSTPD30


## Combining data

In [163]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'old_demo_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'old_demo_score_FPD30_gini',
       'old_demo_score_FSPD30_gini', 'old_demo_score_FSTPD30_gini'],
      dtype=object)

In [164]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'bad_rate','old_demo_score_FPD10_gini','old_demo_score_FPD30_gini',    'old_demo_score_FSPD30_gini', 'old_demo_score_FSTPD30_gini']].copy()

In [165]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_old_demo_score;"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=3a6ce55f-db32-4790-85d1-dcbe07c2a5f9>

In [166]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('old_demo_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('old_demo_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('old_demo_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('old_demo_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_old_demo_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=d2398370-bf9a-41fc-987e-22e1f517bed3>

# bu_bureau_score

In [167]:
sq = """
with bu_bureau_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    bu_bureau_score,
    ln_fpd10_flag,
	ln_mature_fpd10_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd10_flag is not null
  AND
    coalesce(bu_bureau_score, 0.0) > 0.0
  AND
    ln_mature_fpd10_flag = 1
)
select * from bu_bureau_score;
"""

bu_bureau_scorefpd10 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

bu_bureau_scorefpd10.head()

gini_results = calculate_periodic_gini_threedigit(bu_bureau_scorefpd10, 'bu_bureau_score', 'ln_fpd10_flag', 'FPD10')
M1FPD10 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M1FPD10.shape}")
M1FPD10.columns.values

# FPD30

sq = """
with bu_bureau_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    bu_bureau_score,
    ln_fpd30_flag,
	ln_mature_fpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fpd30_flag is not null
  AND
    coalesce(bu_bureau_score, 0.0) > 0.0
  AND
    ln_mature_fpd30_flag = 1
)
select * from bu_bureau_score;
"""

bu_bureau_scorefpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

bu_bureau_scorefpd30.head()

gini_results = calculate_periodic_gini_threedigit(bu_bureau_scorefpd30, 'bu_bureau_score', 'ln_fpd30_flag', 'FPD30')
M2FPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M2FPD30.shape}")
M2FPD30.columns.values

sq = """
with bu_bureau_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    bu_bureau_score,
    ln_fspd30_flag,
	ln_mature_fspd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fspd30_flag is not null
  AND
    coalesce(bu_bureau_score, 0.0) > 0.0
  AND
    ln_mature_fspd30_flag = 1
)
select * from bu_bureau_score;
"""

bu_bureau_scorefspd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(bu_bureau_scorefspd30.head(2))

gini_results = calculate_periodic_gini_threedigit(bu_bureau_scorefspd30, 'bu_bureau_score', 'ln_fspd30_flag', 'FSPD30')
M3FSPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M3FSPD30.shape}")
print(M3FSPD30.columns.values)

M3FSPD30.head()

sq = """
with bu_bureau_score as 
(SELECT 
    ln_disb_dtime disbursementdate,
	digitalLoanAccountId,
    bu_bureau_score,
    ln_fstpd30_flag,
	ln_mature_fstpd30_flag,	
	FROM 
    risk_mart.sil_risk_ds_master_20230101_20250309
  WHERE 
     -- ln_disb_dtime >= '2023-07-01'
    ln_appln_submit_datetime >= '2023-01-01'
  -- AND
  --   format_date('%Y-%m', ln_disb_dtime) = '2024-09'
  AND
    ln_fstpd30_flag is not null
  AND
    coalesce(bu_bureau_score, 0.0) > 0.0
  AND
    ln_mature_fstpd30_flag = 1
)
select * from bu_bureau_score;
"""

bu_bureau_scorefstpd30 = client.query(sq).to_dataframe(progress_bar_type='tqdm')

print(bu_bureau_scorefstpd30.head(2))

gini_results = calculate_periodic_gini_threedigit(bu_bureau_scorefstpd30, 'bu_bureau_score', 'ln_fstpd30_flag', 'FSTPD30')
M4FSTPD30 = gini_results.copy()
print(f"The shape of dataframe after copy is:\t{M4FSTPD30.shape}")
print(M4FSTPD30.columns.values)

M4FSTPD30.head()



Job ID 62c6fc8d-649c-4101-99ed-ce6df3359d18 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(92, 7)
Job ID fa4bf89a-911b-4da8-a667-d094caf7949c successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
The shape of dataframe after copy is:	(92, 7)
Job ID 2be492b6-9f69-4e35-8ca1-9906e9e9c151 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  bu_bureau_score  \
0 2024-03-24 16:15:13  36a51f56-9627-4685-bc0f-c961de6f6d3f            299.0   
1 2023-06-11 18:57:01  1f6d8454-a0d7-45e1-a661-ea6a73fcc16a            347.0   

   ln_fspd30_flag  ln_mature_fspd30_flag  
0               1                      1  
1               1                      1  
The shape of dataframe after copy is:	(92, 7)
['start_date' 'end_date' 'bu_bureau_score_FSPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']
Job ID 34962bc7-1b93-4524-bee1-404fa6fe7e61 successfully executed: 100%|██████████|

d:\Anaconda3\envs\inc_env\lib\site-packages\google\cloud\bigquery\table.py:1820: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(



Downloading: 100%|██████████|
     disbursementdate                  digitalLoanAccountId  bu_bureau_score  \
0 2024-03-24 16:15:13  36a51f56-9627-4685-bc0f-c961de6f6d3f            299.0   
1 2023-06-11 18:57:01  1f6d8454-a0d7-45e1-a661-ea6a73fcc16a            347.0   

   ln_fstpd30_flag  ln_mature_fstpd30_flag  
0                1                       1  
1                1                       1  
The shape of dataframe after copy is:	(92, 7)
['start_date' 'end_date' 'bu_bureau_score_FSTPD30_gini' 'period'
 'Model_Name' 'version' 'bad_rate']


,start_date,end_date,bu_bureau_score_FSTPD30_gini,period,Model_Name,version,bad_rate
0,2023-01-01,2023-01-31,0.050000,Month,bu_bureau_score,1.1.0,FSTPD30
1,2023-01-09,2023-01-15,-0.053333,Week,bu_bureau_score,1.1.0,FSTPD30
2,2023-01-16,2023-01-22,-0.833333,Week,bu_bureau_score,1.1.0,FSTPD30
3,2023-01-23,2023-01-29,0.416667,Week,bu_bureau_score,1.1.0,FSTPD30
4,2023-01-30,2023-02-05,0.244444,Week,bu_bureau_score,1.1.0,FSTPD30


## Combining data

In [168]:
import functools

dataframes = [M1FPD10, M2FPD30, M3FSPD30, M4FSTPD30]
common_columns = ['start_date', 'end_date', 'period', 'Model_Name','version', 'bad_rate']

def merge_dataframes(df1, df2):
    return pd.merge(df1, df2, on=common_columns, how='outer')

final_df = functools.reduce(merge_dataframes, dataframes)

final_df.columns.values

array(['start_date', 'end_date', 'bu_bureau_score_FPD10_gini', 'period',
       'Model_Name', 'version', 'bad_rate', 'bu_bureau_score_FPD30_gini',
       'bu_bureau_score_FSPD30_gini', 'bu_bureau_score_FSTPD30_gini'],
      dtype=object)

In [169]:
final_df = final_df[['start_date', 'end_date', 'period',   'Model_Name', 'version', 'bad_rate','bu_bureau_score_FPD10_gini','bu_bureau_score_FPD30_gini',    'bu_bureau_score_FSPD30_gini', 'bu_bureau_score_FSTPD30_gini']].copy()

In [170]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_bu_bureau_score;"""
client.query(sq)

QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=366ac9cb-21eb-4ff3-a3de-5fa2e67391da>

In [171]:
import pandas as pd
from google.cloud import bigquery

# Create a BigQuery client
client = bigquery.Client('prj-prod-dataplatform')

# Define your table schema
table_schema = [
    bigquery.SchemaField('start_date', 'TIMESTAMP'),
    bigquery.SchemaField('end_date', 'TIMESTAMP'),
    bigquery.SchemaField('period', 'STRING'),
    bigquery.SchemaField('Model_Name', 'STRING'),
    bigquery.SchemaField('version', 'STRING'),
    bigquery.SchemaField('Badrate', 'STRING'),
    bigquery.SchemaField('bu_bureau_score_FPD10_gini', 'FLOAT'),
    bigquery.SchemaField('bu_bureau_score_FPD30_gini', 'FLOAT'),    
    bigquery.SchemaField('bu_bureau_score_FSPD30_gini', 'FLOAT'),
    bigquery.SchemaField('bu_bureau_score_FSTPD30_gini', 'FLOAT')
    
]

# Create your BigQuery table
table_id = 'prj-prod-dataplatform.dap_ds_poweruser_playground.Model_gini_bu_bureau_score'
table = bigquery.Table(table_id, schema=table_schema)
table = client.create_table(table)

# Load your DataFrame into BigQuery
job_config = bigquery.LoadJobConfig(
    write_disposition='WRITE_TRUNCATE'
)

load_job = client.load_table_from_dataframe(
    final_df, table_id, job_config=job_config
)

load_job.result()

LoadJob<project=prj-prod-dataplatform, location=asia-southeast1, id=d3622a49-b785-46ef-af7a-fcb7b2f1eec9>